# HR Interface

The main goal of the project is to perform data comparison between the HR databases of yesterday and today, reveal of updates and sending them to Hilan in specific format required by HILAN ( = events). <br>
Undertaken steps:
1. Load of new data (= "today") from files in blob storage: inbound/datastage/hr/input
2. Comparation of new data with old data (="yesterday") using databricks tables
3. Highlight of gaps and marking of data in line with events criterions for further extraction and sending
4. Event by event, load of information required by HILAN
5. Aggregation of all the events in one massive dataset and split by company name 
6. Concatenation of data with header and footer, encoding it to Windows-1255 (HILAN requirement) and saving it to files in blob storage inbound/datastage/hr/output
7. Update of tables and preparation of data for the next run: load of "new" data to the databricks tables 
8. Moving inbound files to archive

The process it uses, creates or modifies the following tables: <br>

Main tables (refreshed daily): <br>
* `Stg_il_ee_sap_current` – stores data from IL_EE_SAP_CURRENT file for the day 
* `Stg_il_ee_sap_future` – stores data from IL_EE_SAP_FUTURE file for the day 
* `Stg_il_ee_sap_multiple` – stores data from IL_EE_SAP_MULTIPLE file for the day 

Data from files completed with the fields "file_type" and "file_date" and service columns: "Z3_M1", "Z8_I8", "MP2OSEM" that are used in the process  of comparison tables and creating events <br>

Supporting tables  (refreshed daily): 
* `Stg_hilan_all_changes_vw` – aggregation of all the lines from stg_il_ee_sap_current and stg_ee_il_sap_future that contain changes for the day 
* `Stg_fut_curr_yesterday` – aggregation of data from stg_il_ee_sap_current and stg_ee_il_sap_future (split by file type) for the day before current. It is used for comparison process and creation of events. 

DIM tables (constant, not refreshed): 
* `Dim_hilan_branch` -  used to identify the branch depending on sub_area_code 
* `Dim_hilan_contract_status` – status depending on group, subgroup and action_type; used in event 279, 218 
* `Dim_hilan_countries` – converting the sap country code to hilan country code; used in event 554 
* `dim_hilan_existing_id_numbers` – used to exclude certain id numbers from the output 
* `dim_hilan_marital_status` – convert marital status to hilan code; used in event 205 
* `dim_hilan_pay_types` – convert pay grade to hilan pay grade; used in event 572/486 

In [0]:
# import libraries
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, when, concat, substring, lit, split, lower, input_file_name, concat_ws, length
from datetime import datetime
import time

In [0]:
# constants
container = 'prod'
input_location = f'dbfs:/mnt/{container}/inbound/datastage/hr/input/'
output_location = f'dbfs:/mnt/{container}/inbound/datastage/hr/output/'
temp = f'dbfs:/mnt/{container}/inbound/datastage/hr/temp/'
db_name = f'hr_{container}'

In [0]:
schema = StructType([
StructField("PRNR", StringType(), True),
StructField("EMPLOYMENT_STATUS", StringType(), True),
StructField("LAST_NAME", StringType(), True),
StructField("FIRST_NAME", StringType(), True),
StructField("LAST_NAME_HEB", StringType(), True),
StructField("FIRST_NAME_HEB", StringType(), True),
StructField("USER", StringType(), True),
StructField("EMAIL", StringType(), True),
StructField("WORK_TEL", StringType(), True),
StructField("IT_FLAG", StringType(), True),
StructField("CELL", StringType(), True),
StructField("PRIVATE_PHONE", StringType(), True),
StructField("LAST_NAME_SRCH", StringType(), True),
StructField("FIRST_NAME_SRCH", StringType(), True),
StructField("MIDDLE_NAME", StringType(), True),
StructField("FULL_NAME", StringType(), True),
StructField("TITLE", StringType(), True),
StructField("EMP_ID", StringType(), True),
StructField("ID_NUMBER_2", StringType(), True),
StructField("MARITAL_STATUS", StringType(), True),
StructField("MARITAL_STATUS_DATE", StringType(), True),
StructField("BIRTH_DATE", StringType(), True),
StructField("BIRTH_MONTH", StringType(), True),
StructField("BIRTH_YEAR", StringType(), True),
StructField("BIRTH_COUNTRY", StringType(), True),
StructField("BIRTH_DATE_PASS", StringType(), True),
StructField("BIRTH_DATE_MON_YEAR", StringType(), True),
StructField("GENDER", StringType(), True),
StructField("NATIONALITY", StringType(), True),
StructField("LANGUAGE", StringType(), True),
StructField("EMP_MANAGER", StringType(), True),
StructField("EMP_MANAGER_DESC", StringType(), True),
StructField("FUNCTION", StringType(), True),
StructField("FUNCTION_DESC", StringType(), True),
StructField("SUB_FUNCTION", StringType(), True),
StructField("SUB_FUNCTION_DESC", StringType(), True),
StructField("FUNCTION_UNIT", StringType(), True),
StructField("FUNCTION_UNIT_DESC", StringType(), True),
StructField("LOB1", StringType(), True),
StructField("LOB1_DESC", StringType(), True),
StructField("LOB2", StringType(), True),
StructField("LOB2_DESC", StringType(), True),
StructField("PAYROLL_AREA", StringType(), True),
StructField("PAYROLL_AREA_DESC", StringType(), True),
StructField("WORK_CONTRACT", StringType(), True),
StructField("WORK_CONTRACT_DESC", StringType(), True),
StructField("SUBAREA", StringType(), True),
StructField("SUBAREA_DESC", StringType(), True),
StructField("COMPANY_CODE", StringType(), True),
StructField("COMPANY_CODE_DESC", StringType(), True),
StructField("AREA", StringType(), True),
StructField("PERS_AREA_DESC", StringType(), True),
StructField("FORMATED_NAME", StringType(), True),
StructField("COST_CENTER", StringType(), True),
StructField("COST_CENTER_DESC", StringType(), True),
StructField("ORG_UNIT", StringType(), True),
StructField("ORG_UNIT_DESC", StringType(), True),
StructField("GROUP_", StringType(), True),
StructField("GROUP_DESC", StringType(), True),
StructField("SUBGROUP", StringType(), True),
StructField("SUBGROUP_DESC", StringType(), True),
StructField("GE_SUBGROUP", StringType(), True),
StructField("GE_SUBGROUP_DESC", StringType(), True),
StructField("POSITION", StringType(), True),
StructField("PAYROLL_ADMIN", StringType(), True),
StructField("PAYROLL_ADMIN_DESC", StringType(), True),
StructField("PERSADMIN", StringType(), True),
StructField("PERSADMIN_DESC", StringType(), True),
StructField("ADMIN_GROUP", StringType(), True),
StructField("JOB", StringType(), True),
StructField("JOB_DESC", StringType(), True),
StructField("ACTION_REASON", StringType(), True),
StructField("ACTION_REASON_DESC", StringType(), True),
StructField("ACTION_TYPE", StringType(), True),
StructField("ACTION_TYPE_DESC", StringType(), True),
StructField("CUSTOMER_STATUS", StringType(), True),
StructField("CUSTOMER_STATUS_DESC", StringType(), True),
StructField("SPECIAL_PAY_STATUS", StringType(), True),
StructField("SPECIAL_PAY_STATUS_DESC", StringType(), True),
StructField("SUBTYPE", StringType(), True),
StructField("SUBTYPE_DESC", StringType(), True),
StructField("ADDRESS_TYPE", StringType(), True),
StructField("ADDRESS_TYPE_DESC", StringType(), True),
StructField("HOUSE_NUM", StringType(), True),
StructField("ADDRESS_COUNTRY", StringType(), True),
StructField("ADDRESS_CITY", StringType(), True),
StructField("ADDRESS_DISTRICT", StringType(), True),
StructField("ADDRESS_POSTAL_CODE", StringType(), True),
StructField("ADDRESS_CITY_CODE", StringType(), True),
StructField("ADRESS_STREET_HOUSE", StringType(), True),
StructField("TEL_NUM_HR", StringType(), True),
StructField("ADDRESS_SUBTY", StringType(), True),
StructField("ADDRESS_SUBTYPE_DESC", StringType(), True),
StructField("ANNUAL_WORK_HOURS", StringType(), True),
StructField("PART_TIME_EMP", StringType(), True),
StructField("TIME_MANAGE_STATUS", StringType(), True),
StructField("TIME_MANAGE_STATUS_DESC", StringType(), True),
StructField("WORK_SCHED_RULE", StringType(), True),
StructField("WORK_SCHED_RULE_DESC", StringType(), True),
StructField("EMP_PERCENTAGE", StringType(), True),
StructField("BANK_KEY", StringType(), True),
StructField("BANK_ACCOUNT_NUM", StringType(), True),
StructField("BANK_COUNTRY", StringType(), True),
StructField("BANK_CITY", StringType(), True),
StructField("BANK_POSTAL_CODE", StringType(), True),
StructField("BANK_DETAILS_TYPE", StringType(), True),
StructField("BANK_DETAILS_TYPE_DESC", StringType(), True),
StructField("BANK_PAYEE", StringType(), True),
StructField("PAYMENT_CURRENCY", StringType(), True),
StructField("PAYMENT_METHOD", StringType(), True),
StructField("PAYMENT_METHOD_DESC", StringType(), True),
StructField("BANK_SUBTY", StringType(), True),
StructField("CONTRACT_TYPE", StringType(), True),
StructField("CONTRACT_TYPE_DESC", StringType(), True),
StructField("CONTRACT_START", StringType(), True),
StructField("CONTRACT_END_DATE", StringType(), True),
StructField("BUILDING_NUM", StringType(), True),
StructField("ROOM_NUM", StringType(), True),
StructField("COMM_SUBTYPE", StringType(), True),
StructField("SALES_OFFICE", StringType(), True),
StructField("SALES_OFFICE_DESC", StringType(), True),
StructField("SALES_GROUP", StringType(), True),
StructField("SALES_GROUP_DESC", StringType(), True),
StructField("SALES_ORG", StringType(), True),
StructField("SALES_ORG_DESC", StringType(), True),
StructField("DATE_SPEC_TYPE1", StringType(), True),
StructField("DATE_SPEC_TYPE_DESC1", StringType(), True),
StructField("DATE_SPEC1_DATE", StringType(), True),
StructField("DATE_SPEC_TYPE2", StringType(), True),
StructField("DATE_SPEC_TYPE_DESC2", StringType(), True),
StructField("DATE_SPEC2_DATE", StringType(), True),
StructField("DATE_SPEC_TYPE3", StringType(), True),
StructField("DATE_SPEC_TYPE_DESC3", StringType(), True),
StructField("DATE_SPEC3_DATE", StringType(), True),
StructField("DATE_SPEC_TYPE4", StringType(), True),
StructField("DATE_SPEC_TYPE_DESC4", StringType(), True),
StructField("DATE_SPEC4_DATE", StringType(), True),
StructField("DATE_SPEC_TYPE5", StringType(), True),
StructField("DATE_SPEC_TYPE_DESC5", StringType(), True),
StructField("DATE_SPEC5_DATE", StringType(), True),
StructField("DATE_SPEC_TYPE6", StringType(), True),
StructField("DATE_SPEC_TYPE_DESC6", StringType(), True),
StructField("DATE_SPEC6_DATE", StringType(), True),
StructField("DATE_SPEC_TYPE7", StringType(), True),
StructField("DATE_SPEC_TYPE_DESC7", StringType(), True),
StructField("DATE_SPEC7_DATE", StringType(), True),
StructField("DATE_SPEC_TYPE8", StringType(), True),
StructField("DATE_SPEC_TYPE_DESC8", StringType(), True),
StructField("DATE_SPEC8_DATE", StringType(), True),
StructField("DATE_SPEC_TYPE9", StringType(), True),
StructField("DATE_SPEC_TYPE_DESC9", StringType(), True),
StructField("DATE_SPEC9_DATE", StringType(), True),
StructField("DATE_SPEC_TYPE10", StringType(), True),
StructField("DATE_SPEC_TYPE_DESC10", StringType(), True),
StructField("DATE_SPEC10_DATE", StringType(), True),
StructField("BEGDA_0002", StringType(), True),
StructField("ENDDA_0002", StringType(), True),
StructField("CHANGED_PERSON_0002", StringType(), True),
StructField("UPDT_0002", StringType(), True),
StructField("BEGDA_0001", StringType(), True),
StructField("ENDDA_0001", StringType(), True),
StructField("CHANGED_PERSON_0001", StringType(), True),
StructField("UPDT_0001", StringType(), True),
StructField("BEGDA_0000", StringType(), True),
StructField("ENDDA_0000", StringType(), True),
StructField("CHANGED_PERSON_0000", StringType(), True),
StructField("UPDT_0000", StringType(), True),
StructField("BEGDA_0006", StringType(), True),
StructField("ENDDA_0006", StringType(), True),
StructField("CHANGED_PERSON_0006", StringType(), True),
StructField("UPDT_0006", StringType(), True),
StructField("BEGDA_0007", StringType(), True),
StructField("ENDDA_0007", StringType(), True),
StructField("CHANGED_PERSON_0007", StringType(), True),
StructField("UPDT_0007", StringType(), True),
StructField("BEGDA_0009", StringType(), True),
StructField("ENDDA_0009", StringType(), True),
StructField("CHANGED_PERSON_0009", StringType(), True),
StructField("UPDT_0009", StringType(), True),
StructField("BEGDA_0016", StringType(), True),
StructField("ENDDA_0016", StringType(), True),
StructField("CHANGED_PERSON_0016", StringType(), True),
StructField("UPDT_0016", StringType(), True),
StructField("BEGDA_0032", StringType(), True),
StructField("ENDDA_0032", StringType(), True),
StructField("CHANGED_PERSON_0032", StringType(), True),
StructField("UPDT_0032", StringType(), True),
StructField("BEGDA_0900", StringType(), True),
StructField("ENDDA_0900", StringType(), True),
StructField("CHANGED_PERSON_0900", StringType(), True),
StructField("UPDT_0900", StringType(), True),
StructField("USER_BEGDA_0105", StringType(), True),
StructField("USER_ENDDA_0105", StringType(), True),
StructField("USER_CHANGED_PERSON_0105", StringType(), True),
StructField("USER_UPDT_0105", StringType(), True),
StructField("EMAIL_BEGDA_0105", StringType(), True),
StructField("EMAIL_ENDDA_0105", StringType(), True),
StructField("EMAIL_CHANGED_PERSON_0105", StringType(), True),
StructField("EMAIL_UPDT_0105", StringType(), True),
StructField("WORKTEL_BEGDA_0105", StringType(), True),
StructField("WORKTEL_ENDDA_0105", StringType(), True),
StructField("WORKTEL_CHANGED_PERSON_0105", StringType(), True),
StructField("WORKTEL_UPDT_0105", StringType(), True),
StructField("ITFLAG_BEGDA_0105", StringType(), True),
StructField("ITFLAG_ENDDA_0105", StringType(), True),
StructField("ITFLAG_CHANGED_PERSON_0105", StringType(), True),
StructField("ITFLAG_UPDT_0105", StringType(), True),
StructField("CELL_BEGDA_0105", StringType(), True),
StructField("CELL_ENDDA_0105", StringType(), True),
StructField("CELL_CHANGED_PERSON_0105", StringType(), True),
StructField("CELL_UPDT_0105", StringType(), True),
StructField("PRIVATEPHONE_BEGDA_0105", StringType(), True),
StructField("PRIVATEPHONE_ENDDA_0105", StringType(), True),
StructField("PRIVATEPHONE_CHANGED_PR_0105", StringType(), True),
StructField("PRIVATEPHONE_UPDT_0105", StringType(), True),
StructField("BEGDA_0041", StringType(), True),
StructField("ENDDA_0041", StringType(), True),
StructField("CHANGED_PERSON_0041", StringType(), True),
StructField("UPDT_0041", StringType(), True),
StructField("CP", StringType(), True),
StructField("PAY_TYPE", StringType(), True),
StructField("PAY_AREA", StringType(), True),
StructField("PAY_GRADE", StringType(), True),
StructField("BEGDA_0008", StringType(), True),
StructField("ENDDA_0008", StringType(), True),
StructField("CHANGED_PERSON_0008", StringType(), True),
StructField("POSITION_DESC_EN", StringType(), True),
StructField("POSITION_DESC_HEB", StringType(), True),
StructField("COST_CENTER_HEB", StringType(), True),
StructField("ORG_UNIT_HEB", StringType(), True),
StructField("PERS_AREA_HEB", StringType(), True),
StructField("FUNC_UNIT_HEB", StringType(), True),
StructField("FUNC_UNIT_BEGDA", StringType(), True)])

In [0]:
schema_multiple = StructType([
StructField("PRNR", StringType(), True),
StructField("BEGDA_0021", StringType(), True),
StructField("ENDDA_0021", StringType(), True),
StructField("CHANGED_PERSON_0021", StringType(), True),
StructField("UPDT_0021", StringType(), True),
StructField("REF_PRNR_FAMILY_MEMBER", StringType(), True),
StructField("FAMILY_RECORD_TYPE", StringType(), True),
StructField("FAMILY_NATIONALITY", StringType(), True),
StructField("FAMILY_NATIONALITY_2", StringType(), True),
StructField("FAMILY_NATIONALITY_3", StringType(), True),
StructField("FAMILY_LAST_NAME", StringType(), True),
StructField("FAMILY_FIRST_NAME", StringType(), True),
StructField("FAMILY_GENDER", StringType(), True),
StructField("FAMILY_BIRTH_DATE", StringType(), True),
StructField("FAMILY_BIRTH_NAME", StringType(), True)])

In [0]:
files = dbutils.fs.ls(input_location)

In [0]:
if len(files)==0:
  dbutils.notebook.exit("No files found.")
else: 
  print("Files found.")
  display(files)

Files found.


path,name,size,modificationTime
dbfs:/mnt/prod/inbound/datastage/hr/input/IL_EE_SAP_CURRENT_20241231_2000.CSV.DAT,IL_EE_SAP_CURRENT_20241231_2000.CSV.DAT,8447235,1735823233000
dbfs:/mnt/prod/inbound/datastage/hr/input/IL_EE_SAP_FUTURE_20241231_2000.CSV.DAT,IL_EE_SAP_FUTURE_20241231_2000.CSV.DAT,495846,1735823232000
dbfs:/mnt/prod/inbound/datastage/hr/input/IL_EE_SAP_MULTIPLE_20241231_2000.CSV.DAT,IL_EE_SAP_MULTIPLE_20241231_2000.CSV.DAT,983260,1735823232000


In [0]:
# if the files are more than for one day, we select only the files that are the latest
files_count = len(files)
if files_count > 3:
  df_files = spark.createDataFrame(files, ["path", "name", "size", "modificationTime"])
  df_files = df_files.withColumn("date", split(df_files["name"], "_")[4])
  files = df_files.orderBy(df_files["date"].desc()).limit(3).collect()
  display(files)
  files_to_remove = df_files.orderBy(df_files["date"].asc()).limit(files_count-3).collect()
  display(files_to_remove)
  for file in files_to_remove:
    print(f"File {file[1]} moved to archive without processing")
    dbutils.fs.mv(file[0], file[0].replace('inbound', 'archive'))

## Load of files & creation of tables / views

In [0]:
# reading files to datasets
for file in files:
  file_type = file[1].split("_")[3]
  # current & future
  if file_type != "MULTIPLE":
    globals()[f"df_{file_type}"] = spark.read.options(header='false', inferSchema='True', delimiter=';', encoding = "UTF-8").format('csv').load(file[0], schema = schema)
    # add flag columns for further analysis
    globals()[f"df_{file_type}"] = globals()[f"df_{file_type}"].withColumn("Z3_M1", when(((col('ACTION_TYPE')=='Z3')& (col('ACTION_REASON')=='M1')),1).otherwise(0))\
  .withColumn("Z8_I8", when(((col('ACTION_TYPE')=='Z8')& (col('ACTION_REASON')=='I8')),1).otherwise(0))\
    .withColumn("MP2OSEM", when(((col('ACTION_TYPE')=='Z3')& (col('ACTION_REASON')=='M1'))|((col('ACTION_TYPE')=='Z8')& (col('ACTION_REASON')=='I8')),1).otherwise(0))
  # multiple
  else:
    globals()[f"df_{file_type}"] = spark.read.options(header='false', inferSchema='True', delimiter=';', encoding = "UTF-8").format('csv').load(file[0], schema = schema_multiple)
  # add columns file_type and file_date  
  globals()[f"df_{file_type}"] = globals()[f"df_{file_type}"].withColumn("FILE_TYPE", lit(file_type))\
    .withColumn("FILE_DATE", split(input_file_name(), "_")[4])

In [0]:
# COUNT CHECK
count_current_new = df_CURRENT.count()
count_current_old = spark.sql(f"select * from {db_name}.STG_IL_EE_SAP_CURRENT").count()

count_multiple_new = df_MULTIPLE.count()
count_multiple_old= spark.sql(f"select * from {db_name}.STG_IL_EE_SAP_MULTIPLE").count()

check_count_current = abs(1- count_current_new/count_current_old)*100
check_count_multiple = abs(1- count_multiple_new/count_multiple_old)*100

if check_count_current >15:
  dbutils.notebook.exit(f"The file IL_EE_SAP_CURRENT is corrupted or incomplete, delta count is {check_count_current}%")
elif check_count_multiple >15: 
  dbutils.notebook.exit(f"The file IL_EE_SAP_MULTIPLE is corrupted or incomplete, delta count is {check_count_multiple}%")
else:
  print("Check count completed.")

Check count completed.


In [0]:
df_FUTURE.createOrReplaceTempView('FutureNew')
df_CURRENT.createOrReplaceTempView('CurrentNew')
df_MULTIPLE.createOrReplaceTempView('STG_IL_EE_SAP_MULTIPLE_NEW')

In [0]:
stmt_current = f"create or replace temporary view FutureOld as (select * from {db_name}.STG_IL_EE_SAP_FUTURE)"
spark.sql(stmt_current)
stmt_future = f"create or replace temporary view CurrentOld as (select * from {db_name}.STG_IL_EE_SAP_CURRENT)"
spark.sql(stmt_future)

DataFrame[]

In [0]:
# list of columns for automation of SQL queries
list_of_columns = ', '.join([f"n.{col} N_{col}, o.{col} O_{col}" for col in df_CURRENT.columns])
display(list_of_columns)

'n.PRNR N_PRNR, o.PRNR O_PRNR, n.EMPLOYMENT_STATUS N_EMPLOYMENT_STATUS, o.EMPLOYMENT_STATUS O_EMPLOYMENT_STATUS, n.LAST_NAME N_LAST_NAME, o.LAST_NAME O_LAST_NAME, n.FIRST_NAME N_FIRST_NAME, o.FIRST_NAME O_FIRST_NAME, n.LAST_NAME_HEB N_LAST_NAME_HEB, o.LAST_NAME_HEB O_LAST_NAME_HEB, n.FIRST_NAME_HEB N_FIRST_NAME_HEB, o.FIRST_NAME_HEB O_FIRST_NAME_HEB, n.USER N_USER, o.USER O_USER, n.EMAIL N_EMAIL, o.EMAIL O_EMAIL, n.WORK_TEL N_WORK_TEL, o.WORK_TEL O_WORK_TEL, n.IT_FLAG N_IT_FLAG, o.IT_FLAG O_IT_FLAG, n.CELL N_CELL, o.CELL O_CELL, n.PRIVATE_PHONE N_PRIVATE_PHONE, o.PRIVATE_PHONE O_PRIVATE_PHONE, n.LAST_NAME_SRCH N_LAST_NAME_SRCH, o.LAST_NAME_SRCH O_LAST_NAME_SRCH, n.FIRST_NAME_SRCH N_FIRST_NAME_SRCH, o.FIRST_NAME_SRCH O_FIRST_NAME_SRCH, n.MIDDLE_NAME N_MIDDLE_NAME, o.MIDDLE_NAME O_MIDDLE_NAME, n.FULL_NAME N_FULL_NAME, o.FULL_NAME O_FULL_NAME, n.TITLE N_TITLE, o.TITLE O_TITLE, n.EMP_ID N_EMP_ID, o.EMP_ID O_EMP_ID, n.ID_NUMBER_2 N_ID_NUMBER_2, o.ID_NUMBER_2 O_ID_NUMBER_2, n.MARITAL_STATUS 

## Condition for events sending

Each event has a list of triggering actions. If the triggering condition fulfilled, the event should be created. In this case the column with event number with get "1". <br>
The comparison is done specifically for every case: current new to current old, future new to future old, current to future.

In [0]:
# CURRENT NEW TO OLD
events_curr_new_old = '''
--200
case when (o.EMP_ID is null and n.EMP_ID is not null and (n.FIRST_NAME is not null or n.LAST_NAME is not null or n.MIDDLE_NAME is not null or n.BIRTH_DATE is not null or n.BEGDA_0000 is not null or not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1'))) 
              or nvl(n.FIRST_NAME,';')<>nvl(o.FIRST_NAME,';') 
              or nvl(n.LAST_NAME,';')<>nvl(o.LAST_NAME,';') 
              or nvl(n.MIDDLE_NAME,';')<>nvl(o.MIDDLE_NAME,';') 
              or nvl(n.BIRTH_DATE,';')<>nvl(o.BIRTH_DATE,';') 
              or (nvl(n.BEGDA_0000,';')<>nvl(o.BEGDA_0000,';') AND n.ACTION_TYPE='Z1') 
              or o.PRNR is null 
              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fn.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External') then 1 else 0 end Event200,
              --201
              case when (nvl(n.ACTION_TYPE,';')<>nvl(o.ACTION_TYPE,';') 
                              or nvl(n.ACTION_REASON,';')<>nvl(o.ACTION_REASON,';') 
                              or nvl(n.BEGDA_0000,';')<>nvl(o.BEGDA_0000,';')) 
              and n.ACTION_TYPE in ('Z3','Z8') then 1 else 0 end Event201,
              --205
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.MARITAL_STATUS is not null or n.MARITAL_STATUS_DATE is not null or n.GENDER is not null or not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1'))) 
              or nvl(n.MARITAL_STATUS,';')<>nvl(o.MARITAL_STATUS,';') 
              or nvl(n.MARITAL_STATUS_DATE,';')<>nvl(o.MARITAL_STATUS_DATE,';') 
              or nvl(n.GENDER,';')<>nvl(o.GENDER,';') 
              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fn.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External') then 1 else 0 end Event205, 
              --218
              case when (o.EMP_ID is null and n.EMP_ID is not null and (n.ADDRESS_CITY_CODE is not null or n.ADDRESS_POSTAL_CODE is not null or n.ADRESS_STREET_HOUSE is not null or n.PRIVATE_PHONE is not null or n.HOUSE_NUM is not null or not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1'))) 
              or nvl(n.ADDRESS_CITY_CODE,';')<>nvl(o.ADDRESS_CITY_CODE,';') 
              or nvl(n.ADDRESS_POSTAL_CODE,';')<>nvl(o.ADDRESS_POSTAL_CODE,';') 
              or nvl(n.ADRESS_STREET_HOUSE,';')<>nvl(o.ADRESS_STREET_HOUSE,';') 
              or nvl(n.PRIVATE_PHONE,';')<>nvl(o.PRIVATE_PHONE,';') 
              or nvl(n.HOUSE_NUM,';')<>nvl(o.HOUSE_NUM,';') 
              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fn.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External') then 1 else 0 end  Event218,
              --221
              case when (o.EMP_ID is null and n.EMP_ID is not null and (n.ACTION_TYPE is not null or n.ACTION_REASON is not null or n.GROUP_ is not null or n.SUBGROUP is not null))               or nvl(n.ACTION_TYPE,';')<>nvl(o.ACTION_TYPE,';') 
              or nvl(n.ACTION_REASON,';')<>nvl(o.ACTION_REASON,';') 
              or nvl(n.GROUP_,';')<>nvl(o.GROUP_,';')  
              or nvl(n.SUBGROUP,';')<>nvl(o.SUBGROUP,';')then 1 else 0 end Event221, 
              --260
              case when (o.EMP_ID is null and n.EMP_ID is not null and (n.ACTION_TYPE is not null or n.ACTION_REASON is not null or n.BEGDA_0000 is not null)) 
              or (nvl(n.ACTION_TYPE,';')<>nvl(o.ACTION_TYPE,';') 
                  or nvl(n.ACTION_REASON,';')<>nvl(o.ACTION_REASON,';') 
                  or nvl(n.BEGDA_0000,';')<>nvl(o.BEGDA_0000,';') 
                  or nvl(n.SUBGROUP,';')<>nvl(o.SUBGROUP,';'))
              and n.ACTION_TYPE in ('Z1','Z3') then 1 else 0 end Event260, 
              --274
              case when (nvl(n.ACTION_TYPE,';')<>nvl(o.ACTION_TYPE,';') 
              or nvl(n.ACTION_REASON,';')<>nvl(o.ACTION_REASON,';')
              or nvl(n.BEGDA_0000,';')<>nvl(o.BEGDA_0000,';')) 
              and n.ACTION_TYPE in ('Z7','Z6') then 1 else 0 end Event274, 
              --278 OSEM Just FunctionUnit
              case when (nvl(n.FUNCTION_UNIT,';')<>nvl(o.FUNCTION_UNIT,';') AND not (n.ACTION_TYPE='Z8' and n.FUNCTION_UNIT is null))
                              and not (nvl(n.SUBAREA,';')<>nvl(o.SUBAREA,';') AND not (n.ACTION_TYPE='Z8' and n.SUBAREA is null))
                              and not (nvl(n.COST_CENTER,';')<>nvl(o.COST_CENTER,';') AND not (n.ACTION_TYPE='Z8' and n.COST_CENTER is null))
                              and not (nvl(n.POSITION,';')<>nvl(o.POSITION,';') AND not (n.ACTION_TYPE='Z8' and nvl(n.POSITION,'99999999') ='99999999'))
                              and not (nvl(n.PRNR,';')<>nvl(o.PRNR,';') AND not (n.ACTION_TYPE='Z8' and n.PRNR is null)) 
                              and not (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fn.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External') 
                              AND not (n.ACTION_TYPE='Z8' and n.SUBGROUP is null)
                              then 1 else 0 end Event278_JustFunctionUnit, 
              --278 OSEM
              case when (o.EMP_ID is null and n.EMP_ID is not null and (n.FUNCTION_UNIT is not null or n.SUBAREA is not null or n.COST_CENTER is not null or n.POSITION is not null or  not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1'))) 
                              or (nvl(n.FUNCTION_UNIT,';')<>nvl(o.FUNCTION_UNIT,';') AND not (n.ACTION_TYPE='Z8' and n.FUNCTION_UNIT is null))
                              or(nvl(n.SUBAREA,';')<>nvl(o.SUBAREA,';') AND not (n.ACTION_TYPE='Z8' and n.SUBAREA is null))
                              or(nvl(n.COST_CENTER,';')<>nvl(o.COST_CENTER,';') AND not (n.ACTION_TYPE='Z8' and n.COST_CENTER is null))
                              or(nvl(n.POSITION,';')<>nvl(o.POSITION,';') AND not (n.ACTION_TYPE='Z8' and nvl(n.POSITION,'99999999') ='99999999'))
                              or(nvl(n.PRNR,';')<>nvl(o.PRNR,';') AND not (n.ACTION_TYPE='Z8' and n.PRNR is null)) 
                              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fn.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External')then 1 else 0 end Event278_OSEM, 
              --278 NSP
              case when (o.EMP_ID is null and n.EMP_ID is not null and (n.SUBAREA is not null or not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1'))) 
                        or nvl(n.SUBAREA,';')<>nvl(o.SUBAREA,';') AND not (n.ACTION_TYPE='Z8' and n.SUBAREA is null) 
                        or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fn.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External') then 1 else 0 end Event278_NSP, 
              --279
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.GROUP_ is not null or n.SUBGROUP is not null)) 
              or nvl(n.GROUP_,';')<>nvl(o.GROUP_,';') 
              or nvl(n.SUBGROUP,';')<>nvl(o.SUBGROUP,';') 
              or (n.ACTION_TYPE='Z3' and o.ACTION_TYPE<>'Z3' ) then 1 else 0 end Event279, 
              --280
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.EMP_PERCENTAGE is not null or not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1'))) 
              or nvl(n.EMP_PERCENTAGE,';')<>nvl(o.EMP_PERCENTAGE,';') 
              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fn.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External') then 1 else 0 end Event280,
              --298
              case when ((o.EMP_ID is null and n.EMP_ID is not null and n.EMAIL is not null) or nvl(n.EMAIL,';')<>nvl(o.EMAIL,';')) AND n.EMAIL is not null AND n.ACTION_TYPE<>'Z8' then 1 else 0 end  Event298,
              --303
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.BANK_KEY is not null or n.BANK_ACCOUNT_NUM is not null)) 
              or nvl(n.BANK_KEY,';')<>nvl(o.BANK_KEY,';') 
              or nvl(n.BANK_ACCOUNT_NUM,';')<>nvl(o.BANK_ACCOUNT_NUM,';') then 1 else 0 end Event303, 
              --551
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.FIRST_NAME is not null or n.LAST_NAME is not null or n.MIDDLE_NAME is not null)) 
              or nvl(n.FIRST_NAME,';')<>nvl(o.FIRST_NAME,';') 
              or nvl(n.LAST_NAME,';')<>nvl(o.LAST_NAME,';') 
              or nvl(n.MIDDLE_NAME,';')<>nvl(o.MIDDLE_NAME,';') then 1 else 0 end Event551, 
              --552
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.FIRST_NAME_HEB is not null or n.LAST_NAME_HEB is not null or not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1'))) 
              or nvl(n.FIRST_NAME_HEB,';')<>nvl(o.FIRST_NAME_HEB,';') 
              or nvl(n.LAST_NAME_HEB,';')<>nvl(o.LAST_NAME_HEB,';') 
              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fn.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External')then 1 else 0 end Event552, 
              --554
              case when nvl(n.BIRTH_DATE_PASS,';')<>nvl(o.BIRTH_DATE_PASS,';') 
              or nvl(n.BIRTH_COUNTRY,';')<>nvl(o.BIRTH_COUNTRY,';') 
              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fn.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External') then 1 else 0 end Event554, 
              0 Event558, --case when nvl(n.JOB,';')<>nvl(o.JOB,';') then 1 else 0 end 
              --572/305
              case when ((nvl(n.ACTION_TYPE,';')<>nvl(o.ACTION_TYPE,';') or nvl(n.BEGDA_0000,';')<>nvl(o.BEGDA_0000,';')) and n.ACTION_TYPE ='Z8') 
              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fn.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External') then 1 else 0 end Event572_305, 
             --572/460
              case when (o.PRNR is null and n.AREA='ILN1')
              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fn.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External') then 1 else 0 end Event572_460, 
              --572/471
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.JOB is not null or not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1'))) 
              or nvl(n.JOB,';')<>nvl(o.JOB,';') 
              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fn.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External')then 1 else 0 end Event572_471, 
              --572/486
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.PAY_TYPE is not null or n.PAY_AREA is not null or n.PAY_GRADE is not null or not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1'))) 
                          or (nvl(n.PAY_TYPE,';')<>nvl(o.PAY_TYPE,';') AND not (n.ACTION_TYPE='Z8' and n.PAY_TYPE is null))
                          or(nvl(n.PAY_AREA,';')<>nvl(o.PAY_AREA,';') AND not (n.ACTION_TYPE='Z8' and n.PAY_AREA is null))
                          or(nvl(n.PAY_GRADE,';')<>nvl(o.PAY_GRADE,';')AND not (n.ACTION_TYPE='Z8' and n.PAY_GRADE is null)) 
                          or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fn.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' and n.PAY_GRADE is null)then 1 else 0 end Event572_486, 
              --572/541
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.CP is not null or not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1'))) 
              or nvl(n.CP,';')<>nvl(o.CP,';') 
              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fn.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External') then 1 else 0 end Event572_541, 
              --572/560
              case when (o.EMP_ID is null and n.EMP_ID is not null and n.EMP_MANAGER is not null)
                        or (nvl(n.EMP_MANAGER,';')<>nvl(o.EMP_MANAGER,';') AND not (n.ACTION_TYPE='Z8' and n.EMP_MANAGER is null))
                        or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fn.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External') then 1 else 0 end Event572_560, 
              --800/150
              case when (o.EMP_ID is null and n.EMP_ID is not null and n.COST_CENTER is not null) or nvl(n.COST_CENTER,';')<>nvl(o.COST_CENTER,';') then 1 else 0 end Event800_150, '''

In [0]:
# FUTURE TO CURRENT
events_fut_curr = '''
--200
case when (o.EMP_ID is null and n.EMP_ID is not null and (n.FIRST_NAME is not null or n.LAST_NAME is not null or n.MIDDLE_NAME is not null or n.BIRTH_DATE is not null or n.BEGDA_0000 is not null)) 
              or nvl(n.FIRST_NAME,';')<>nvl(o.FIRST_NAME,';') 
              or nvl(n.LAST_NAME,';')<>nvl(o.LAST_NAME,';') 
              or nvl(n.MIDDLE_NAME,';')<>nvl(o.MIDDLE_NAME,';') 
              or nvl(n.BIRTH_DATE,';')<>nvl(o.BIRTH_DATE,';') 
              or (nvl(n.BEGDA_0000,';')<>nvl(o.BEGDA_0000,';') AND n.ACTION_TYPE='Z1') 
              or o.PRNR is null 
              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) then 1 else 0 end Event200,
              --201
              case when ((nvl(n.ACTION_TYPE,';')<>nvl(o.ACTION_TYPE,';') 
                              or nvl(n.ACTION_REASON,';')<>nvl(o.ACTION_REASON,';') 
                              or nvl(n.BEGDA_0000,';')<>nvl(o.BEGDA_0000,';')) and n.ACTION_TYPE in ('Z3','Z8')) 
                              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' )
              then 1 else 0 end Event201,
              --205
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.MARITAL_STATUS is not null or n.MARITAL_STATUS_DATE is not null or n.GENDER is not null)) 
                              or nvl(n.MARITAL_STATUS,';')<>nvl(o.MARITAL_STATUS,';') 
                              or nvl(n.MARITAL_STATUS_DATE,';')<>nvl(o.MARITAL_STATUS_DATE,';') 
                              or nvl(n.GENDER,';')<>nvl(o.GENDER,';') 
                              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) then 1 else 0 end Event205, 
              --218
              case when (o.EMP_ID is null and n.EMP_ID is not null and (n.ADDRESS_CITY_CODE is not null or n.ADDRESS_POSTAL_CODE is not null or n.ADRESS_STREET_HOUSE is not null or n.PRIVATE_PHONE is not null or n.HOUSE_NUM is not null)) 
                              or nvl(n.ADDRESS_CITY_CODE,';')<>nvl(o.ADDRESS_CITY_CODE,';')
                              or nvl(n.ADDRESS_POSTAL_CODE,';')<>nvl(o.ADDRESS_POSTAL_CODE,';')
                              or nvl(n.ADRESS_STREET_HOUSE,';')<>nvl(o.ADRESS_STREET_HOUSE,';')
                              or nvl(n.PRIVATE_PHONE,';')<>nvl(o.PRIVATE_PHONE,';')
                              or nvl(n.HOUSE_NUM,';')<>nvl(o.HOUSE_NUM,';')  
                              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' )
              then 1 else 0 end Event218,
              --221
              case when (o.EMP_ID is null and n.EMP_ID is not null and (n.ACTION_TYPE is not null or n.ACTION_REASON is not null or n.GROUP_ is not null or n.SUBGROUP is not null)) or nvl(n.ACTION_TYPE,';')<>nvl(o.ACTION_TYPE,';') or nvl(n.ACTION_REASON,';')<>nvl(o.ACTION_REASON,';')  or nvl(n.GROUP_,';')<>nvl(o.GROUP_,';') or nvl(n.SUBGROUP,';')<>nvl(o.SUBGROUP,';') or nvl(n.SUBGROUP,';')<>nvl(o.SUBGROUP,';')then 1 else 0 end Event221, 
              --260
              case when (((o.EMP_ID is null and n.EMP_ID is not null and (n.ACTION_TYPE is not null or n.ACTION_REASON is not null or n.BEGDA_0000 is not null)) or (nvl(n.ACTION_TYPE,';')<>nvl(o.ACTION_TYPE,';') or nvl(n.ACTION_REASON,';')<>nvl(o.ACTION_REASON,';') or nvl(n.BEGDA_0000,';')<>nvl(o.BEGDA_0000,';') or nvl(n.SUBGROUP,';')<>nvl(o.SUBGROUP,';'))) and n.ACTION_TYPE in ('Z1','Z3'))                              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) then 1 else 0 end Event260, 
              --274
              case when (nvl(n.ACTION_TYPE,';')<>nvl(o.ACTION_TYPE,';') or nvl(n.ACTION_REASON,';')<>nvl(o.ACTION_REASON,';')or nvl(n.BEGDA_0000,';')<>nvl(o.BEGDA_0000,';')) and n.ACTION_TYPE in ('Z7','Z6') then 1 else 0 end Event274, 
              --278 OSEM Just FunctionUnit
              case when (nvl(n.FUNCTION_UNIT,';')<>nvl(o.FUNCTION_UNIT,';') AND not (n.ACTION_TYPE='Z8' and n.FUNCTION_UNIT is null))
                              and not (nvl(n.SUBAREA,';')<>nvl(o.SUBAREA,';') AND not (n.ACTION_TYPE='Z8' and n.SUBAREA is null))
                              and not (nvl(n.COST_CENTER,';')<>nvl(o.COST_CENTER,';') AND not (n.ACTION_TYPE='Z8' and n.COST_CENTER is null))
                              and not (nvl(n.POSITION,';')<>nvl(o.POSITION,';') AND not (n.ACTION_TYPE='Z8' and nvl(n.POSITION,'99999999') ='99999999'))
                              and not (nvl(n.PRNR,';')<>nvl(o.PRNR,';') AND not (n.ACTION_TYPE='Z8' and n.PRNR is null)) 
                              and not (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' 
                              and n.GROUP_DESC<>'External' )then 1 else 0 end Event278_JustFunctionUnit, 
             --278 OSEM
              case when (o.EMP_ID is null and n.EMP_ID is not null and (n.FUNCTION_UNIT is not null or n.SUBAREA is not null or n.COST_CENTER is not null or n.POSITION is not null)) 
                              or (nvl(n.SUBAREA,';')<>nvl(o.SUBAREA,';') AND not (n.ACTION_TYPE='Z8' and n.SUBAREA is null))
                              or(nvl(n.FUNCTION_UNIT,';')<>nvl(o.FUNCTION_UNIT,';') AND not (n.ACTION_TYPE='Z8' and n.FUNCTION_UNIT is null))
                              or(nvl(n.COST_CENTER,';')<>nvl(o.COST_CENTER,';') AND not (n.ACTION_TYPE='Z8' and n.COST_CENTER is null))
                              or(nvl(n.POSITION,';')<>nvl(o.POSITION,';') AND not (n.ACTION_TYPE='Z8' and nvl(n.POSITION,'99999999') ='99999999'))
                              or(nvl(n.PRNR,';')<>nvl(o.PRNR,';') AND not (n.ACTION_TYPE='Z8' and n.PRNR is null)) 
                              or((n.ACTION_TYPE='Z3' and n.ACTION_REASON='M1' and not(nvl(o.ACTION_TYPE,';')='Z3' and nvl(o.ACTION_REASON,';')='M1')) AND not (n.ACTION_TYPE='Z8' and n.SUBGROUP is null)) 
                              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) then 1 else 0 end Event278_OSEM, 
              --278 NSP
              case when (o.EMP_ID is null and n.EMP_ID is not null and n.SUBAREA is not null) 
              or nvl(n.SUBAREA,';')<>nvl(o.SUBAREA,';') AND not (n.ACTION_TYPE='Z8' and n.SUBAREA is null) 
              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' )
              then 1 else 0 end Event278_NSP, 
              --279
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.GROUP_ is not null or n.SUBGROUP is not null)) 
              or nvl(n.GROUP_,';')<>nvl(o.GROUP_,';') 
              or nvl(n.SUBGROUP,';')<>nvl(o.SUBGROUP,';') 
              or (n.ACTION_TYPE='Z3' and o.ACTION_TYPE<>'Z3' ) 
              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) 
              then 1 else 0 end Event279, 
              --280
              case when (o.EMP_ID is null and n.EMP_ID is not null and n.EMP_PERCENTAGE is not null) 
              or nvl(n.EMP_PERCENTAGE,';')<>nvl(o.EMP_PERCENTAGE,';')
              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' )
              then 1 else 0 end Event280,
              --298
              case when (o.EMP_ID is null and n.EMP_ID is not null and n.EMAIL is not null) or nvl(n.EMAIL,';')<>nvl(o.EMAIL,';') AND nvl(n.EMAIL,';')<>';' AND n.ACTION_TYPE<>'Z8' then 1 else 0 end  Event298,
              --303
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.BANK_KEY is not null or n.BANK_ACCOUNT_NUM is not null)) 
                              or nvl(n.BANK_KEY,';')<>nvl(o.BANK_KEY,';') 
                              or nvl(n.BANK_ACCOUNT_NUM,';')<>nvl(o.BANK_ACCOUNT_NUM,';') 
                              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' )
              then 1 else 0 end Event303, 
              --551
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.FIRST_NAME is not null or n.LAST_NAME is not null or n.MIDDLE_NAME is not null)) 
                              or nvl(n.FIRST_NAME,';')<>nvl(o.FIRST_NAME,';') 
                              or nvl(n.LAST_NAME,';')<>nvl(o.LAST_NAME,';') 
                              or nvl(n.MIDDLE_NAME,';')<>nvl(o.MIDDLE_NAME,';') 
                              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' )
              then 1 else 0 end Event551, 
              --552
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.FIRST_NAME_HEB is not null or n.LAST_NAME_HEB is not null)) 
                            or nvl(n.FIRST_NAME_HEB,';')<>nvl(o.FIRST_NAME_HEB,';') 
                            or nvl(n.LAST_NAME_HEB,';')<>nvl(o.LAST_NAME_HEB,';')  
                            or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) then 1 else 0 end Event552, 
              --554
              case when nvl(n.BIRTH_DATE_PASS,';')<>nvl(o.BIRTH_DATE_PASS,';') 
                              or nvl(n.BIRTH_COUNTRY,';')<>nvl(o.BIRTH_COUNTRY,';')  
                              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) then 1 else 0 end Event554, 
              0 as Event558, 
              --572/305
              case when ((nvl(n.ACTION_TYPE,';')<>nvl(o.ACTION_TYPE,';') or nvl(n.BEGDA_0000,';')<>nvl(o.BEGDA_0000,';')) and n.ACTION_TYPE ='Z8') or (n.ACTION_TYPE='Z3' and n.ACTION_REASON='M1' and not(nvl(o.ACTION_TYPE,';')='Z3' and nvl(o.ACTION_REASON,';')='M1'))then 1 else 0 end Event572_305, 
              --572/460
              case when (o.PRNR is null and n.AREA='ILN1') then 1 else 0 end Event572_460, 
              --572/471
              case when (o.EMP_ID is null and n.EMP_ID is not null and n.JOB is not null) 
                              or nvl(n.JOB,';')<>nvl(o.JOB,';')  
                              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' )
              then 1 else 0 end Event572_471, 
              --572/486
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.PAY_TYPE is not null or n.PAY_AREA is not null or n.PAY_GRADE is not null)) 
                              or (nvl(n.PAY_TYPE,';')<>nvl(o.PAY_TYPE,';') AND not (n.ACTION_TYPE='Z8' and n.PAY_TYPE is null))
                              or(nvl(n.PAY_AREA,';')<>nvl(o.PAY_AREA,';') AND not (n.ACTION_TYPE='Z8' and n.PAY_AREA is null))
                              or(nvl(n.PAY_GRADE,';')<>nvl(o.PAY_GRADE,';')AND not (n.ACTION_TYPE='Z8' and n.PAY_GRADE is null)) 
                              or ((n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External')  and n.PAY_GRADE is null)
              then 1 else 0 end Event572_486, 
              --572/541
              case when (o.EMP_ID is null and n.EMP_ID is not null and n.CP is not null) 
                            or nvl(n.CP,';')<>nvl(o.CP,';')
                            or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External') then 1 else 0 end Event572_541, 
              --572/560
              case when (o.EMP_ID is null and n.EMP_ID is not null and n.EMP_MANAGER is not null) 
                        or (nvl(n.EMP_MANAGER,';')<>nvl(o.EMP_MANAGER,';') AND not (n.ACTION_TYPE='Z8' and n.EMP_MANAGER is null))
                        or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External') then 1 else 0 end Event572_560, 
              --800/150
              case when (o.EMP_ID is null and n.EMP_ID is not null and n.COST_CENTER is not null) or nvl(n.COST_CENTER,';')<>nvl(o.COST_CENTER,';') then 1 else 0 end Event800_150, 
'''

In [0]:
# FUTURE OLD TO NEW
events_fut_old_new = '''
              --200
              case when (o.EMP_ID is null and n.EMP_ID is not null and (n.FIRST_NAME is not null or n.LAST_NAME is not null or n.MIDDLE_NAME is not null or n.BIRTH_DATE is not null or n.BEGDA_0000 is not null or (not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1')))) 
                      or nvl(n.FIRST_NAME,';')<>nvl(o.FIRST_NAME,';') 
                      or nvl(n.LAST_NAME,';')<>nvl(o.LAST_NAME,';') 
                      or nvl(n.MIDDLE_NAME,';')<>nvl(o.MIDDLE_NAME,';')
                      or nvl(n.BIRTH_DATE,';')<>nvl(o.BIRTH_DATE,';') 
                      or (nvl(n.BEGDA_0000,';')<>nvl(o.BEGDA_0000,';')  AND n.ACTION_TYPE='Z1') 
                      or o.PRNR is null 
                      or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(cn.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) then 1 else 0 end Event200,
              --201
              case when (nvl(n.ACTION_TYPE,';')<>nvl(o.ACTION_TYPE,';')  or nvl(n.ACTION_REASON,';')<>nvl(o.ACTION_REASON,';') or nvl(n.BEGDA_0000,';')<>nvl(o.BEGDA_0000,';')) and n.ACTION_TYPE in ('Z3','Z8') then 1 else 0 end Event201,
              --205
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.MARITAL_STATUS is not null or n.MARITAL_STATUS_DATE is not null or n.GENDER is not null or  (not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1')))) 
                              or nvl(n.MARITAL_STATUS,';')<>nvl(o.MARITAL_STATUS,';') 
                              or nvl(n.MARITAL_STATUS_DATE,';')<>nvl(o.MARITAL_STATUS_DATE,';')
                              or nvl(n.GENDER,';')<>nvl(o.GENDER,';') 
                              or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(cn.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) then 1 else 0 end Event205,
              --218
              case when (o.EMP_ID is null and n.EMP_ID is not null and (n.ADDRESS_CITY_CODE is not null or n.ADDRESS_POSTAL_CODE is not null or n.ADRESS_STREET_HOUSE is not null or n.PRIVATE_PHONE is not null or n.HOUSE_NUM is not null or  (not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1')))) 
                            or nvl(n.ADDRESS_CITY_CODE,';')<>nvl(o.ADDRESS_CITY_CODE,';') 
                            or nvl(n.ADDRESS_POSTAL_CODE,';')<>nvl(o.ADDRESS_POSTAL_CODE,';') 
                            or nvl(n.ADRESS_STREET_HOUSE,';')<>nvl(o.ADRESS_STREET_HOUSE,';') 
                            or nvl(n.PRIVATE_PHONE,';')<>nvl(o.PRIVATE_PHONE,';') 
                            or nvl(n.HOUSE_NUM,';')<>nvl(o.HOUSE_NUM,';') 
                            or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(cn.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' )then 1 else 0 end Event218,
              --221
              case when (o.EMP_ID is null and n.EMP_ID is not null and (n.ACTION_TYPE is not null or n.ACTION_REASON is not null or n.GROUP_ is not null or n.SUBGROUP is not null))                             
              or nvl(n.ACTION_TYPE,';')<>nvl(o.ACTION_TYPE,';')                             
              or nvl(n.ACTION_REASON,';')<>nvl(o.ACTION_REASON,';')                              
              or nvl(n.GROUP_,';')<>nvl(o.GROUP_,';')                             
              or nvl(n.SUBGROUP,';')<>nvl(o.SUBGROUP,';')then 1 else 0 end Event221, 
              --260
              case when (o.EMP_ID is null and n.EMP_ID is not null and (n.ACTION_TYPE is not null or n.ACTION_REASON is not null or n.BEGDA_0000 is not null))               
              or (nvl(n.ACTION_TYPE,';')<>nvl(o.ACTION_TYPE,';')               
              or nvl(n.ACTION_REASON,';')<>nvl(o.ACTION_REASON,';')               
              or nvl(n.BEGDA_0000,';')<>nvl(o.BEGDA_0000,';')               
              or nvl(n.SUBGROUP,';')<>nvl(o.SUBGROUP,';'))and n.ACTION_TYPE in ('Z1','Z3') then 1 else 0 end Event260, 
              --274
              case when (nvl(n.ACTION_TYPE,';')<>nvl(o.ACTION_TYPE,';') or nvl(n.ACTION_REASON,';')<>nvl(o.ACTION_REASON,';')or nvl(n.BEGDA_0000,';')<>nvl(o.BEGDA_0000,';')) and n.ACTION_TYPE in ('Z7','Z6') then 1 else 0 end Event274, 
              --278 OSEM Just FunctionUnit
              case when (o.EMP_ID is null and n.EMP_ID is not null and (n.FUNCTION_UNIT is not null or n.SUBAREA is not null or n.COST_CENTER is not null or n.POSITION is not null or  (not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1')))) 
                              or (nvl(n.FUNCTION_UNIT,';')<>nvl(o.FUNCTION_UNIT,';') AND not (n.ACTION_TYPE='Z8' and n.FUNCTION_UNIT is null))
                              and not (nvl(n.SUBAREA,';')<>nvl(o.SUBAREA,';') AND not (n.ACTION_TYPE='Z8' and n.SUBAREA is null))
                              and not (nvl(n.COST_CENTER,';')<>nvl(o.COST_CENTER,';') AND not (n.ACTION_TYPE='Z8' and n.COST_CENTER is null))
                              and not (nvl(n.POSITION,';')<>nvl(o.POSITION,';') AND not (n.ACTION_TYPE='Z8' and nvl(n.POSITION,'99999999') ='99999999'))
                              and not (nvl(n.PRNR,';')<>nvl(o.PRNR,';') AND not (n.ACTION_TYPE='Z8' and n.PRNR is null)) 
                              and not (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(cn.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' 
                              and n.GROUP_DESC<>'External' ) then 1 else 0 end Event278_JustFunctionUnit, 
              --278 OSEM
              case when (nvl(n.SUBAREA,';')<>nvl(o.SUBAREA,';') AND not (n.ACTION_TYPE='Z8' and n.SUBAREA is null))
                              or(nvl(n.FUNCTION_UNIT,';')<>nvl(o.FUNCTION_UNIT,';') AND not (n.ACTION_TYPE='Z8' and n.FUNCTION_UNIT is null))
                              or(nvl(n.COST_CENTER,';')<>nvl(o.COST_CENTER,';') AND not (n.ACTION_TYPE='Z8' and n.COST_CENTER is null))
                              or(nvl(n.POSITION,';')<>nvl(o.POSITION,';') AND not (n.ACTION_TYPE='Z8' and nvl(n.POSITION,'99999999') ='99999999'))
                              or(nvl(n.PRNR,';')<>nvl(o.PRNR,';') AND not (n.ACTION_TYPE='Z8' and n.PRNR is null)) 
                              or(n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(cn.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) then 1 else 0 end Event278_OSEM, 
              --278 NSP
              case when (o.EMP_ID is null and n.EMP_ID is not null and (n.SUBAREA is not null or  (not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1')))) 
                        or nvl(n.SUBAREA,';')<>nvl(o.SUBAREA,';') AND not (n.ACTION_TYPE='Z8' and n.SUBAREA is null) 
                        or(n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(cn.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) then 1 else 0 end Event278_NSP, 
              --279
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.GROUP_ is not null or n.SUBGROUP is not null)) 
                              or nvl(n.GROUP_,';')<>nvl(o.GROUP_,';') 
                              or nvl(n.SUBGROUP,';')<>nvl(o.SUBGROUP,';') 
                              or (n.ACTION_TYPE='Z3' and o.ACTION_TYPE<>'Z3' ) then 1 else 0 end Event279,
              --280
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.EMP_PERCENTAGE is not null or  (not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1')))) 
                            or nvl(n.EMP_PERCENTAGE,';')<>nvl(o.EMP_PERCENTAGE,';') 
                            or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(cn.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) then 1 else 0 end Event280,
              --298
              case when (o.EMP_ID is null and n.EMP_ID is not null and n.EMAIL is not null) or (nvl(n.EMAIL,';')<>nvl(o.EMAIL,';') AND nvl(n.EMAIL,';')<>';' AND n.ACTION_TYPE<>'Z8') then 1 else 0 end  Event298,
              --303
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.BANK_KEY is not null or n.BANK_ACCOUNT_NUM is not null)) 
                            or nvl(n.BANK_KEY,';')<>nvl(o.BANK_KEY,';') 
                            or nvl(n.BANK_ACCOUNT_NUM,';')<>nvl(o.BANK_ACCOUNT_NUM,';') then 1 else 0 end Event303, 
              --551
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.FIRST_NAME is not null or n.LAST_NAME is not null or n.MIDDLE_NAME is not null)) 
                            or nvl(n.FIRST_NAME,';')<>nvl(o.FIRST_NAME,';')
                            or nvl(n.LAST_NAME,';')<>nvl(o.LAST_NAME,';') 
                            or nvl(n.MIDDLE_NAME,';')<>nvl(o.MIDDLE_NAME,';') then 1 else 0 end Event551,
              --552
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.FIRST_NAME_HEB is not null or n.LAST_NAME_HEB is not null or  (not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1')))) 
                            or nvl(n.FIRST_NAME_HEB,';')<>nvl(o.FIRST_NAME_HEB,';') 
                            or nvl(n.LAST_NAME_HEB,';')<>nvl(o.LAST_NAME_HEB,';') 
                            or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(cn.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) then 1 else 0 end Event552, 
              --554
              case when nvl(n.BIRTH_DATE_PASS,';')<>nvl(o.BIRTH_DATE_PASS,';') 
                            or nvl(n.BIRTH_COUNTRY,';')<>nvl(o.BIRTH_COUNTRY,';') 
                            or(n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(cn.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' )then 1 else 0 end Event554, 
              0 Event558, --case when nvl(n.JOB,';')<>nvl(o.JOB,';') then 1 else 0 end 
              --572/305
              case when ((nvl(n.ACTION_TYPE,';')<>nvl(o.ACTION_TYPE,';') or nvl(n.BEGDA_0000,';')<>nvl(o.BEGDA_0000,';')) and n.ACTION_TYPE ='Z8') 
                            or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(cn.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) then 1 else 0 end Event572_305, 
              --572/460
              case when (o.PRNR is null and n.AREA='ILN1')
                            or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(cn.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) then 1 else 0 end Event572_460, 
              --572/471
              case when o.EMP_ID is null and n.EMP_ID is not null and(n.JOB is not null or nvl(n.JOB,';')<>nvl(o.JOB,';')) 
                            or(n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(cn.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' )then 1 else 0 end Event572_471, 
              --572/486
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.PAY_TYPE is not null or n.PAY_AREA is not null or n.PAY_GRADE is not null or  (not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1')))) 
                            or (nvl(n.PAY_TYPE,';')<>nvl(o.PAY_TYPE,';') AND not (n.ACTION_TYPE='Z8' and n.PAY_TYPE is null))
                            or(nvl(n.PAY_GRADE,';')<>nvl(o.PAY_GRADE,';')AND not (n.ACTION_TYPE='Z8' and n.PAY_GRADE is null)) 
                            or((n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(cn.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) and n.PAY_GRADE is null) then 1 else 0 end Event572_486, 
              --572/541
              case when (o.EMP_ID is null and n.EMP_ID is not null and(n.CP is not null or  (not(nvl(n.ACTION_TYPE,';')='Z3' and nvl(n.ACTION_REASON,';')='M1')))) 
                            or nvl(n.CP,';')<>nvl(o.CP,';') 
                            or (n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(cn.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' ) then 1 else 0 end Event572_541, 
              --572/560
              case when (o.EMP_ID is null and n.EMP_ID is not null and n.EMP_MANAGER is not null)
                            or (nvl(n.EMP_MANAGER,';')<>nvl(o.EMP_MANAGER,';') AND not (n.ACTION_TYPE='Z8' and n.EMP_MANAGER is null))
                            or((((n.Z3_M1=1 and o.Z3_M1=0) or (n.Z8_I8=1 and o.Z8_I8=0)) and n.SUBGROUP<>'IB' ) AND not (n.ACTION_TYPE='Z8' and n.SUBGROUP is null))then 1 else 0 end Event572_560,
              --800/150
              case when (o.EMP_ID is null and n.EMP_ID is not null and n.COST_CENTER is not null) or nvl(n.COST_CENTER,';')<>nvl(o.COST_CENTER,';') then 1 else 0 end Event800_150,
'''

## Aggregation of all the changes together

In [0]:
# STG_HILAN_CURR_NEW_OLD_VW
stmt_curr_new_old = f'''
CREATE OR REPLACE TEMPORARY VIEW STG_HILAN_CURR_NEW_OLD_VW AS
select * from (
select case when o.PRNR is null then 'I' when n.PRNR is null then 'D' else 'U' end UpdateType,
              case when n.AREA<>'ILN1' and n.SUBGROUP <> 'ZY' then 'OSEM' 
              when n.AREA <> 'ILN1' and n.SUBGROUP = 'ZY' then 'PLSTNOSEM' else 'NESPRESSO' end CompanyName,
              case when n.ACTION_TYPE in('Z1','Z3') and n.GROUP_=1 then 1 else 0 end Z1_Z3_Type,
              date_format(add_months(to_date(n.FILE_DATE,'yyyyMMdd'),-1),'MMyyyy') PREV_MONTH,
              date_format(add_months(to_date(n.FILE_DATE,'yyyyMMdd'),-0),'MMyyyy') CURR_MONTH,
case when n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fn.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' then 1 else 0 end MP2OSEM, 
{events_curr_new_old}
              date_format(add_months(to_date(n.BEGDA_0000,'yyyyMMdd'),-1),'ddMMyyyy')Prev_BEGDA_0000,
              case when nvl(n.ADDRESS_CITY_CODE,';')=nvl(o.ADDRESS_CITY_CODE,';') and nvl(n.ADDRESS_POSTAL_CODE,';')=nvl(o.ADDRESS_POSTAL_CODE,';') and nvl(n.ADRESS_STREET_HOUSE,';')=nvl(o.ADRESS_STREET_HOUSE,';') and nvl(n.PRIVATE_PHONE,';')<>nvl(o.PRIVATE_PHONE,';') and nvl(n.HOUSE_NUM,';')=nvl(o.HOUSE_NUM,';')  then 1 else 0 end JustPhone218,
{list_of_columns}
from CurrentNew n
left join CurrentOld o on n.PRNR=o.PRNR 
left join (select * from FutureNew where substr(FILE_DATE,7,2)<>17) fn on n.PRNR=fn.PRNR
left join FutureOld fo on n.PRNR=fo.PRNR) AllEmployees
where 1 in (EVENT200,EVENT201,EVENT205,EVENT218,EVENT221,EVENT260,EVENT274,EVENT278_OSEM ,EVENT278_NSP,EVENT279,EVENT280,EVENT298,EVENT303,EVENT551,EVENT552,EVENT554,EVENT558,EVENT572_305,EVENT572_460,EVENT572_471,EVENT572_486,EVENT572_541,EVENT572_560,EVENT800_150)
'''
spark.sql(stmt_curr_new_old)

DataFrame[]

In [0]:
# STG_HILAN_FUTURE_CURR_VW
stmt_fut_curr = f'''
CREATE OR REPLACE TEMPORARY VIEW STG_HILAN_FUTURE_CURR_VW AS
select * from (
select case when o.PRNR is null then 'I' when n.PRNR is null then 'D' else 'U' end UpdateType,
              case when n.AREA<>'ILN1' and n.SUBGROUP <> 'ZY' then 'OSEM' 
              when n.AREA <> 'ILN1' and n.SUBGROUP = 'ZY' then 'PLSTNOSEM' else 'NESPRESSO' end CompanyName,
              case when n.ACTION_TYPE in('Z1','Z3') and n.GROUP_=1 then 1 else 0 end Z1_Z3_Type,
              date_format(add_months(to_date(n.FILE_DATE,'yyyyMMdd'),-1),'MMyyyy') PREV_MONTH,
              date_format(add_months(to_date(n.FILE_DATE,'yyyyMMdd'),-0),'MMyyyy')  CURR_MONTH,
case when n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(fo.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' then 1 else 0 end MP2OSEM,
 
{events_fut_curr}
              date_format(add_months(to_date(n.BEGDA_0000,'yyyyMMdd'),-1),'ddMMyyyy')Prev_BEGDA_0000,
              case when nvl(n.ADDRESS_CITY_CODE,';')=nvl(o.ADDRESS_CITY_CODE,';') and nvl(n.ADDRESS_POSTAL_CODE,';')=nvl(o.ADDRESS_POSTAL_CODE,';') and nvl(n.ADRESS_STREET_HOUSE,';')=nvl(o.ADRESS_STREET_HOUSE,';') and nvl(n.PRIVATE_PHONE,';')<>nvl(o.PRIVATE_PHONE,';') and nvl(n.HOUSE_NUM,';')=nvl(o.HOUSE_NUM,';')  then 1 else 0 end JustPhone218,
{list_of_columns}
from FutureNew n
left join CurrentNew o on n.PRNR=o.PRNR
left join FutureOld fo on n.PRNR=fo.PRNR
left join CurrentOld co on n.PRNR=co.PRNR) as AllEmployees
where 1 in (EVENT200,EVENT201,EVENT205,EVENT218,EVENT221,EVENT260,EVENT274,EVENT278_OSEM,EVENT278_NSP,EVENT279,EVENT280,EVENT298,EVENT303,EVENT551,EVENT552,EVENT554,EVENT558,EVENT572_305,EVENT572_460,EVENT572_471,EVENT572_486,EVENT572_541,EVENT572_560,EVENT800_150)
'''
spark.sql(stmt_fut_curr)

DataFrame[]

In [0]:
# STG_HILAN_FUTURE_OLD_NEW_VW
stmt_fut_old_new = f'''
CREATE OR REPLACE TEMPORARY VIEW STG_HILAN_FUTURE_OLD_NEW_VW AS
select * from (
select case when o.PRNR is null then 'I' when n.PRNR is null then 'D' else 'U' end UpdateType,
              case when n.AREA<>'ILN1' and n.SUBGROUP <> 'ZY' then 'OSEM' 
              when n.AREA <> 'ILN1' and n.SUBGROUP = 'ZY' then 'PLSTNOSEM' else 'NESPRESSO' end CompanyName,
              case when n.ACTION_TYPE in('Z1','Z3') and n.GROUP_=1 then 1 else 0 end Z1_Z3_Type,
              date_format(add_months(to_date(n.FILE_DATE,'yyyyMMdd'),-1),'MMyyyy') PREV_MONTH,
              date_format(add_months(to_date(n.FILE_DATE,'yyyyMMdd'),-0),'MMyyyy')  CURR_MONTH,
              case when n.MP2OSEM=1 and nvl(o.MP2OSEM,0)=0 and nvl(cn.MP2OSEM,0)=0 and nvl(co.MP2OSEM,0)=0 and n.SUBGROUP<>'IB' and n.GROUP_DESC<>'External' then 1 else 0 end MP2OSEM,
{events_fut_old_new}
              date_format(add_months(to_date(n.BEGDA_0000,'yyyyMMdd'),-1),'ddMMyyyy')Prev_BEGDA_0000,
              case when nvl(n.ADDRESS_CITY_CODE,';')=nvl(o.ADDRESS_CITY_CODE,';') and nvl(n.ADDRESS_POSTAL_CODE,';')=nvl(o.ADDRESS_POSTAL_CODE,';') and nvl(n.ADRESS_STREET_HOUSE,';')=nvl(o.ADRESS_STREET_HOUSE,';') and nvl(n.PRIVATE_PHONE,';')<>nvl(o.PRIVATE_PHONE,';') and nvl(n.HOUSE_NUM,';')=nvl(o.HOUSE_NUM,';')  then 1 else 0 end JustPhone218,
{list_of_columns}
from FutureNew n 
left join FutureOld o on n.PRNR=o.PRNR
left join CurrentOld cn on n.PRNR=cn.PRNR
left join CurrentOld co on n.PRNR=co.PRNR) as AllEmployees
where 1 in (EVENT200,EVENT201,EVENT205,EVENT218,EVENT221,EVENT260,EVENT274,EVENT278_OSEM ,EVENT278_NSP,EVENT279,EVENT280,EVENT298,EVENT303,EVENT551,EVENT552,EVENT554,EVENT558,EVENT572_305,EVENT572_460,EVENT572_471,EVENT572_486,EVENT572_541,EVENT572_560,EVENT800_150)
'''
spark.sql(stmt_fut_old_new)

DataFrame[]

In [0]:
stmt=f'''
CREATE OR REPLACE TABLE {db_name}.STG_HILAN_ALL_CHANGES_VW AS 
with CURR_NEW_OLD as(
        select 'Current New-Old' SourceType,a.*
        from STG_HILAN_CURR_NEW_OLD_VW a
        where not exists (select 1 from {db_name}.STG_HILAN_FUT_CURR_YESTERDAY b where a.N_PRNR=b.N_PRNR and b.DATATYPE='CURR')),
FUTURE_CURR as (
        select 'Future-Current' SourceType, b.*
        from STG_HILAN_FUTURE_CURR_VW b
        where not exists (select 1 from STG_HILAN_CURR_NEW_OLD_VW a where a.N_PRNR=b.N_PRNR)
        and not exists (select 1 from {db_name}.STG_HILAN_FUT_CURR_YESTERDAY c
                                        where c.N_PRNR=b.N_PRNR
                                        and c.DATATYPE='FUTURE')),
FUTURE_OLD_NEW as (
        select 'Future New-Old' SourceType,c.*
        from STG_HILAN_FUTURE_OLD_NEW_VW c
        where not exists (select 1 from STG_HILAN_CURR_NEW_OLD_VW a where a.N_PRNR=c.N_PRNR)
        and not exists (select 1 from STG_HILAN_FUTURE_CURR_VW b where b.N_PRNR=c.N_PRNR and b.n_action_type=b.o_action_type and b.n_begda_0000<>b.o_begda_0000)),    
UnionAll as
(
select * from CURR_NEW_OLD
union all
select * from FUTURE_CURR 
where not exists (select 1 from CURR_NEW_OLD where CURR_NEW_OLD.N_PRNR=FUTURE_CURR.N_PRNR)
union all
select *
from FUTURE_OLD_NEW
where not exists (select 1 from CURR_NEW_OLD where CURR_NEW_OLD.N_PRNR=FUTURE_OLD_NEW.N_PRNR)
      and not exists (select 1 from FUTURE_CURR where FUTURE_CURR.N_PRNR=FUTURE_OLD_NEW.N_PRNR)
)
select SOURCETYPE,N_FILE_DATE, DATE_FORMAT(ADD_MONTHS(TO_DATE(N_FILE_DATE,'yyyyMMdd'),-3),'ddMMyyyy') AS FILE_DATE_3_MONTHS,UPDATETYPE,COMPANYNAME, 
case when d.emp_id is NOT null or N_ACTION_REASON = "02" THEN "1" ELSE "0" END as ExistingID, --- NEW
Z1_Z3_TYPE,PREV_MONTH,CURR_MONTH,MP2OSEM,EVENT200,EVENT201,EVENT205,EVENT218,EVENT221,EVENT260,EVENT274,EVENT278_JUSTFUNCTIONUNIT,EVENT278_OSEM,EVENT278_NSP,EVENT279,EVENT280,EVENT298,EVENT303,EVENT551,EVENT552,EVENT554,EVENT558,EVENT572_305,EVENT572_460,EVENT572_471,EVENT572_486,EVENT572_541,EVENT572_560,EVENT800_150,PREV_BEGDA_0000,JUSTPHONE218,NVL(N_ACTION_REASON, " ") AS N_ACTION_REASON,N_ACTION_REASON_DESC,NVL(N_ACTION_TYPE, " ") AS N_ACTION_TYPE,N_ACTION_TYPE_DESC,N_ADDRESS_CITY,N_ADDRESS_CITY_CODE,N_ADDRESS_COUNTRY,N_ADDRESS_DISTRICT,NVL(N_ADDRESS_POSTAL_CODE, "0") AS N_ADDRESS_POSTAL_CODE,N_ADDRESS_SUBTY,N_ADDRESS_SUBTYPE_DESC,N_ADDRESS_TYPE,N_ADDRESS_TYPE_DESC,N_ADMIN_GROUP,NVL(N_ADRESS_STREET_HOUSE, " ") AS N_ADRESS_STREET_HOUSE,N_ANNUAL_WORK_HOURS,N_AREA,N_BANK_ACCOUNT_NUM,N_BANK_CITY,N_BANK_COUNTRY,NVL(DATE_FORMAT(TO_DATE(N_BEGDA_0000, 'yyyyMMdd'), 'ddMMyyyy'), "00000000") AS N_BEGDA_0000,NVL(DATE_FORMAT(TO_DATE(N_BEGDA_0001, 'yyyyMMdd'), 'ddMMyyyy'),"00000000")AS N_BEGDA_0001,NVL(DATE_FORMAT(TO_DATE(N_BEGDA_0002, 'yyyyMMdd'), 'ddMMyyyy'), "00000000")AS N_BEGDA_0002,NVL(DATE_FORMAT(TO_DATE(N_BEGDA_0006, 'yyyyMMdd'), 'ddMMyyyy'), "00000000")AS N_BEGDA_0006,NVL(DATE_FORMAT(TO_DATE(N_BEGDA_0007, 'yyyyMMdd'), 'ddMMyyyy'), "00000000")AS N_BEGDA_0007,N_BIRTH_COUNTRY,N_BIRTH_DATE,NVL(N_BIRTH_DATE_MON_YEAR, "00") AS N_BIRTH_DATE_MON_YEAR,NVL(N_BIRTH_DATE_PASS, "00000000") AS N_BIRTH_DATE_PASS,NVL(N_BIRTH_MONTH, "00") N_BIRTH_MONTH,NVL(N_BIRTH_YEAR, "0000") N_BIRTH_YEAR,N_BUILDING_NUM,N_COMPANY_CODE,N_COMPANY_CODE_DESC,N_CONTRACT_END_DATE,N_CONTRACT_START,N_CONTRACT_TYPE,N_CONTRACT_TYPE_DESC,NVL(N_COST_CENTER, " ") AS N_COST_CENTER,N_COST_CENTER_DESC,N_COST_CENTER_HEB,N_CP,NVL(N_EMAIL, " ") N_EMAIL, NVL(N_EMAIL_BEGDA_0105, "00000000") AS N_EMAIL_BEGDA_0105,N_EMAIL_UPDT_0105,NVL(N_EMP_ID, 0) AS N_EMP_ID,NVL(N_EMP_MANAGER, "0") N_EMP_MANAGER,N_EMP_MANAGER_DESC,N_EMP_PERCENTAGE,N_EMPLOYMENT_STATUS,NVL(N_FIRST_NAME, " ") AS N_FIRST_NAME,NVL(N_FIRST_NAME_HEB, " ") N_FIRST_NAME_HEB,N_FULL_NAME,N_GE_SUBGROUP,N_GE_SUBGROUP_DESC,NVL(N_GENDER, "0") AS N_GENDER,NVL(N_GROUP_, "0") AS N_GROUP_,N_GROUP_DESC, 
CASE 
WHEN N_ACTION_TYPE = 'Z3' THEN '00'
WHEN N_ACTION_TYPE = 'Z8' AND N_ACTION_REASON ='I8' AND (N_GROUP_ = 9 AND N_SUBGROUP = 'IA') THEN '00'
WHEN N_ACTION_TYPE ='Z8' AND N_ACTION_REASON ='02' THEN '21'
ELSE '20'
END AS STATUS, ---NEW
NVL(N_HOUSE_NUM, "") AS N_HOUSE_NUM,N_ID_NUMBER_2, NVL(N_LAST_NAME, " ") AS N_LAST_NAME, NVL(N_LAST_NAME_HEB, " ") AS N_LAST_NAME_HEB, N_MARITAL_STATUS,NVL(N_MARITAL_STATUS_DATE, "00000000") AS N_MARITAL_STATUS_DATE,NVL(N_MIDDLE_NAME, " ") AS N_MIDDLE_NAME,N_NATIONALITY,N_ORG_UNIT,N_ORG_UNIT_DESC,N_ORG_UNIT_HEB,N_PART_TIME_EMP,N_PAY_AREA, NVL(N_FUNCTION_UNIT, " ") AS N_FUNCTION_UNIT, N_FUNC_UNIT_BEGDA,N_PAY_GRADE,N_PAY_TYPE, N_BANK_KEY, N_JOB,
CASE WHEN COMPANYNAME = 'NESPRESSO' THEN '2282' WHEN N_PERSADMIN IS NULL THEN "0000" ELSE LPAD(N_PERSADMIN, 4, "0") END AS N_PERSADMIN,
N_PERSADMIN_DESC,N_POSITION,N_POSITION_DESC_EN,N_POSITION_DESC_HEB,nvl(N_PRIVATE_PHONE, " 
                   ") as N_PRIVATE_PHONE, 
N_PRIVATEPHONE_BEGDA_0105,N_PRNR,N_SUB_FUNCTION,N_SUB_FUNCTION_DESC,NVL(N_SUBAREA, "0") AS N_SUBAREA,N_SUBAREA_DESC,NVL(N_SUBGROUP, " ") AS N_SUBGROUP,N_SUBGROUP_DESC,N_SUBTYPE,N_SUBTYPE_DESC, NVL(N_TEL_NUM_HR, "0000000000") AS N_TEL_NUM_HR, N_USER,N_USER_BEGDA_0105,NVL(O_ACTION_REASON, " ") AS O_ACTION_REASON,O_ACTION_REASON_DESC,NVL(O_ACTION_TYPE, " ") AS O_ACTION_TYPE,O_ACTION_TYPE_DESC,O_ADDRESS_CITY,O_ADDRESS_CITY_CODE,O_ADDRESS_COUNTRY,O_ADDRESS_DISTRICT,O_ADDRESS_POSTAL_CODE,O_ADDRESS_SUBTY,O_ADDRESS_SUBTYPE_DESC,O_ADDRESS_TYPE,O_ADDRESS_TYPE_DESC,O_ADMIN_GROUP,NVL(O_ADRESS_STREET_HOUSE, " ") AS O_ADRESS_STREET_HOUSE,O_ANNUAL_WORK_HOURS,O_AREA,O_BANK_ACCOUNT_NUM,O_BANK_CITY,O_BANK_COUNTRY,O_BANK_DETAILS_TYPE,O_BANK_DETAILS_TYPE_DESC,O_BANK_KEY,O_BANK_PAYEE,O_BANK_POSTAL_CODE,O_BANK_SUBTY,O_BEGDA_0000,O_BEGDA_0001,O_BEGDA_0002,O_BEGDA_0006,O_BEGDA_0007,O_BIRTH_COUNTRY,O_BIRTH_DATE,O_BIRTH_DATE_MON_YEAR,O_BIRTH_DATE_PASS,O_BIRTH_MONTH,O_BIRTH_YEAR,O_BUILDING_NUM,O_COMPANY_CODE,O_COMPANY_CODE_DESC,O_CONTRACT_END_DATE,O_CONTRACT_START,O_CONTRACT_TYPE,O_CONTRACT_TYPE_DESC,O_COST_CENTER,O_COST_CENTER_DESC,O_COST_CENTER_HEB,O_CP,O_EMAIL,O_EMAIL_ENDDA_0105,O_EMAIL_UPDT_0105,O_EMP_ID,O_EMP_MANAGER,O_EMP_MANAGER_DESC,O_EMP_PERCENTAGE,O_EMPLOYMENT_STATUS,O_FIRST_NAME,O_FIRST_NAME_HEB,O_FIRST_NAME_SRCH,O_FORMATED_NAME,O_FULL_NAME,O_LAST_NAME, O_LAST_NAME_HEB, O_GE_SUBGROUP,O_GE_SUBGROUP_DESC,NVL(O_GENDER, "0") AS O_GENDER,O_GROUP_,O_GROUP_DESC,O_HOUSE_NUM,O_ID_NUMBER_2,O_MARITAL_STATUS,NVL(O_MARITAL_STATUS_DATE, "00000000") AS O_MARITAL_STATUS_DATE,O_MIDDLE_NAME,O_NATIONALITY,O_ORG_UNIT,O_ORG_UNIT_DESC,O_ORG_UNIT_HEB,O_PART_TIME_EMP,O_PAY_AREA,O_PAY_GRADE,O_PAY_TYPE,O_PERSADMIN,O_PERSADMIN_DESC,O_POSITION,O_POSITION_DESC_EN,O_POSITION_DESC_HEB,O_PRIVATE_PHONE,O_PRIVATEPHONE_BEGDA_0105,O_PRIVATEPHONE_CHANGED_PR_0105,O_PRIVATEPHONE_ENDDA_0105,O_PRIVATEPHONE_UPDT_0105,O_PRNR,O_ROOM_NUM,O_SALES_GROUP,O_SALES_GROUP_DESC,O_SALES_OFFICE,O_SALES_OFFICE_DESC,O_SALES_ORG,O_SALES_ORG_DESC,O_SPECIAL_PAY_STATUS,O_SPECIAL_PAY_STATUS_DESC,O_SUB_FUNCTION,O_SUB_FUNCTION_DESC,NVL(O_SUBAREA, "0") AS O_SUBAREA,O_SUBAREA_DESC,O_SUBGROUP,O_SUBGROUP_DESC,O_SUBTYPE,O_SUBTYPE_DESC,O_USER,O_USER_BEGDA_0105, O_FUNCTION_UNIT, O_FUNC_UNIT_BEGDA, O_JOB
FROM UnionAll
left join {db_name}.dim_hilan_existing_id_numbers d on d.emp_id = UnionAll.n_emp_id ---new
WHERE UPDATETYPE <> "D" AND N_EMP_ID IS NOT NULL AND N_AREA NOT IN ("ILC9", "ILC1", "ILC2", "ILC3");
'''
spark.sql(stmt)

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
stmt=f'''
CREATE OR REPLACE TEMPORARY VIEW STG_HILAN_MULTIPLE_CHANGES_VW  (PRNR, REF_PRNR_FAMILY_MEMBER, FAMILY_BIRTH_DATE, FAMILY_FIRST_NAME, FAMILY_GENDER, FAMILY_LAST_NAME, FAMILY_RECORD_TYPE, BEGDA_0021, RELATION_CODE, OLD_PRNR, OLD_REF_MEMBER, COMPANYNAME, FIRST_NAME_HEB, LAST_NAME_HEB, CP, EMP_ID, PERSADMIN) AS 
  with CURRENT_NEW as(
                select nvl(a.PRNR,b.PRNR) PRNR,
                              case 
                              when nvl(a.AREA, b.AREA)='ILN1' THEN 'NESPRESSO'
                              else 'OSEM' end CompanyName,
                              nvl(a.FIRST_NAME_HEB,b.FIRST_NAME_HEB) FIRST_NAME_HEB,
                              nvl(a.LAST_NAME_HEB,b.LAST_NAME_HEB) LAST_NAME_HEB,
                              nvl(a.CP,b.CP) CP,
                              nvl(a.EMP_ID,b.EMP_ID) EMP_ID,
                              nvl(a.PERSADMIN,b.PERSADMIN) PERSADMIN
                from CurrentNew a full join FutureNew b
                on a.PRNR=b.PRNR)
SELECT  distinct nvl(n.prnr, o.prnr) prnr,
                nvl(n.ref_prnr_family_member, o.ref_prnr_family_member) ref_prnr_family_member,
                nvl(n.family_birth_date, o.family_birth_date) family_birth_date,
                nvl(n.family_first_name,o.family_first_name)  family_first_name,
                nvl(n.family_gender,o.family_gender) family_gender,
                nvl(n.family_last_name,o.family_last_name) family_last_name,
                nvl(n.family_record_type,o.family_record_type) family_record_type,
                nvl(n.begda_0021,o.begda_0021) begda_0021,
                case when nvl(n.family_record_type,o.family_record_type)='1' then '1' when nvl(n.family_record_type,o.family_record_type)='2' and nvl(n.family_gender,o.family_gender) in ('1','M') then '2' when nvl(n.family_record_type,o.family_record_type)='2' and nvl(n.family_gender,o.family_gender) in ('2','F') then '3' else ' ' end RELATION_CODE,
                o.prnr OLD_PRNR,
                o.ref_prnr_family_member OLD_REF_MEMBER,
                CompanyName,
                a.FIRST_NAME_HEB,
                a.LAST_NAME_HEB,
                a.CP, 
                a.EMP_ID,
                CASE WHEN COMPANYNAME = 'NESPRESSO' THEN '2282' 
                WHEN a.PERSADMIN IS NULL THEN "0000" 
                ELSE LPAD(a.PERSADMIN, 4, "0") 
                END AS PERSADMIN
FROM  STG_IL_EE_SAP_MULTIPLE_NEW n 
full join {db_name}.STG_IL_EE_SAP_MULTIPLE o
on n.PRNR=o.PRNR
and n.REF_PRNR_FAMILY_MEMBER=o.REF_PRNR_FAMILY_MEMBER
join CURRENT_NEW a
on a.PRNR=nvl(n.prnr, o.prnr) 
WHERE n.REF_PRNR_FAMILY_MEMBER is not null
and  (nvl(n.family_birth_date,';')<> nvl(o.family_birth_date,';')
or nvl(n.family_first_name,';')<> nvl(o.family_first_name,';')
or  nvl(n.family_gender,';')<> nvl(o.family_gender,';')
or  nvl(n.family_last_name,';')<> nvl(o.family_last_name,';')
or  nvl(n.family_record_type,';')<> nvl(o.family_record_type,';')
or  nvl(n.begda_0021,';')<> nvl(o.begda_0021,';'));
'''
spark.sql(stmt)

DataFrame[]

In [0]:
%sql
SELECT * FROM hr_prod.stg_hilan_all_changes_vw

SOURCETYPE,N_FILE_DATE,FILE_DATE_3_MONTHS,UPDATETYPE,COMPANYNAME,ExistingID,Z1_Z3_TYPE,PREV_MONTH,CURR_MONTH,MP2OSEM,EVENT200,EVENT201,EVENT205,EVENT218,EVENT221,EVENT260,EVENT274,EVENT278_JUSTFUNCTIONUNIT,EVENT278_OSEM,EVENT278_NSP,EVENT279,EVENT280,EVENT298,EVENT303,EVENT551,EVENT552,EVENT554,EVENT558,EVENT572_305,EVENT572_460,EVENT572_471,EVENT572_486,EVENT572_541,EVENT572_560,EVENT800_150,PREV_BEGDA_0000,JUSTPHONE218,N_ACTION_REASON,N_ACTION_REASON_DESC,N_ACTION_TYPE,N_ACTION_TYPE_DESC,N_ADDRESS_CITY,N_ADDRESS_CITY_CODE,N_ADDRESS_COUNTRY,N_ADDRESS_DISTRICT,N_ADDRESS_POSTAL_CODE,N_ADDRESS_SUBTY,N_ADDRESS_SUBTYPE_DESC,N_ADDRESS_TYPE,N_ADDRESS_TYPE_DESC,N_ADMIN_GROUP,N_ADRESS_STREET_HOUSE,N_ANNUAL_WORK_HOURS,N_AREA,N_BANK_ACCOUNT_NUM,N_BANK_CITY,N_BANK_COUNTRY,N_BEGDA_0000,N_BEGDA_0001,N_BEGDA_0002,N_BEGDA_0006,N_BEGDA_0007,N_BIRTH_COUNTRY,N_BIRTH_DATE,N_BIRTH_DATE_MON_YEAR,N_BIRTH_DATE_PASS,N_BIRTH_MONTH,N_BIRTH_YEAR,N_BUILDING_NUM,N_COMPANY_CODE,N_COMPANY_CODE_DESC,N_CONTRACT_END_DATE,N_CONTRACT_START,N_CONTRACT_TYPE,N_CONTRACT_TYPE_DESC,N_COST_CENTER,N_COST_CENTER_DESC,N_COST_CENTER_HEB,N_CP,N_EMAIL,N_EMAIL_BEGDA_0105,N_EMAIL_UPDT_0105,N_EMP_ID,N_EMP_MANAGER,N_EMP_MANAGER_DESC,N_EMP_PERCENTAGE,N_EMPLOYMENT_STATUS,N_FIRST_NAME,N_FIRST_NAME_HEB,N_FULL_NAME,N_GE_SUBGROUP,N_GE_SUBGROUP_DESC,N_GENDER,N_GROUP_,N_GROUP_DESC,STATUS,N_HOUSE_NUM,N_ID_NUMBER_2,N_LAST_NAME,N_LAST_NAME_HEB,N_MARITAL_STATUS,N_MARITAL_STATUS_DATE,N_MIDDLE_NAME,N_NATIONALITY,N_ORG_UNIT,N_ORG_UNIT_DESC,N_ORG_UNIT_HEB,N_PART_TIME_EMP,N_PAY_AREA,N_FUNCTION_UNIT,N_FUNC_UNIT_BEGDA,N_PAY_GRADE,N_PAY_TYPE,N_BANK_KEY,N_JOB,N_PERSADMIN,N_PERSADMIN_DESC,N_POSITION,N_POSITION_DESC_EN,N_POSITION_DESC_HEB,N_PRIVATE_PHONE,N_PRIVATEPHONE_BEGDA_0105,N_PRNR,N_SUB_FUNCTION,N_SUB_FUNCTION_DESC,N_SUBAREA,N_SUBAREA_DESC,N_SUBGROUP,N_SUBGROUP_DESC,N_SUBTYPE,N_SUBTYPE_DESC,N_TEL_NUM_HR,N_USER,N_USER_BEGDA_0105,O_ACTION_REASON,O_ACTION_REASON_DESC,O_ACTION_TYPE,O_ACTION_TYPE_DESC,O_ADDRESS_CITY,O_ADDRESS_CITY_CODE,O_ADDRESS_COUNTRY,O_ADDRESS_DISTRICT,O_ADDRESS_POSTAL_CODE,O_ADDRESS_SUBTY,O_ADDRESS_SUBTYPE_DESC,O_ADDRESS_TYPE,O_ADDRESS_TYPE_DESC,O_ADMIN_GROUP,O_ADRESS_STREET_HOUSE,O_ANNUAL_WORK_HOURS,O_AREA,O_BANK_ACCOUNT_NUM,O_BANK_CITY,O_BANK_COUNTRY,O_BANK_DETAILS_TYPE,O_BANK_DETAILS_TYPE_DESC,O_BANK_KEY,O_BANK_PAYEE,O_BANK_POSTAL_CODE,O_BANK_SUBTY,O_BEGDA_0000,O_BEGDA_0001,O_BEGDA_0002,O_BEGDA_0006,O_BEGDA_0007,O_BIRTH_COUNTRY,O_BIRTH_DATE,O_BIRTH_DATE_MON_YEAR,O_BIRTH_DATE_PASS,O_BIRTH_MONTH,O_BIRTH_YEAR,O_BUILDING_NUM,O_COMPANY_CODE,O_COMPANY_CODE_DESC,O_CONTRACT_END_DATE,O_CONTRACT_START,O_CONTRACT_TYPE,O_CONTRACT_TYPE_DESC,O_COST_CENTER,O_COST_CENTER_DESC,O_COST_CENTER_HEB,O_CP,O_EMAIL,O_EMAIL_ENDDA_0105,O_EMAIL_UPDT_0105,O_EMP_ID,O_EMP_MANAGER,O_EMP_MANAGER_DESC,O_EMP_PERCENTAGE,O_EMPLOYMENT_STATUS,O_FIRST_NAME,O_FIRST_NAME_HEB,O_FIRST_NAME_SRCH,O_FORMATED_NAME,O_FULL_NAME,O_LAST_NAME,O_LAST_NAME_HEB,O_GE_SUBGROUP,O_GE_SUBGROUP_DESC,O_GENDER,O_GROUP_,O_GROUP_DESC,O_HOUSE_NUM,O_ID_NUMBER_2,O_MARITAL_STATUS,O_MARITAL_STATUS_DATE,O_MIDDLE_NAME,O_NATIONALITY,O_ORG_UNIT,O_ORG_UNIT_DESC,O_ORG_UNIT_HEB,O_PART_TIME_EMP,O_PAY_AREA,O_PAY_GRADE,O_PAY_TYPE,O_PERSADMIN,O_PERSADMIN_DESC,O_POSITION,O_POSITION_DESC_EN,O_POSITION_DESC_HEB,O_PRIVATE_PHONE,O_PRIVATEPHONE_BEGDA_0105,O_PRIVATEPHONE_CHANGED_PR_0105,O_PRIVATEPHONE_ENDDA_0105,O_PRIVATEPHONE_UPDT_0105,O_PRNR,O_ROOM_NUM,O_SALES_GROUP,O_SALES_GROUP_DESC,O_SALES_OFFICE,O_SALES_OFFICE_DESC,O_SALES_ORG,O_SALES_ORG_DESC,O_SPECIAL_PAY_STATUS,O_SPECIAL_PAY_STATUS_DESC,O_SUB_FUNCTION,O_SUB_FUNCTION_DESC,O_SUBAREA,O_SUBAREA_DESC,O_SUBGROUP,O_SUBGROUP_DESC,O_SUBTYPE,O_SUBTYPE_DESC,O_USER,O_USER_BEGDA_0105,O_FUNCTION_UNIT,O_FUNC_UNIT_BEGDA,O_JOB
Current New-Old,20241231,30092024,U,OSEM,0,1,112024,122024,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,20072019,0,01,New to Nestle,Z1,Hire,קריית מלאכי,001034,IL,null,8305000,1,Permanent residence,1,Permanent residence,IL01,שרת משה,1339.20,ILB3,2402030,קריית מלאכי,IL,20082019,01022022,0505196

## Event 200

In [0]:
event200 =f'''
Select COMPANYNAME, "200", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER,
RPAD(LEFT(N_LAST_NAME_HEB,10), 10, " ") AS L_NAME_SHORT,
RPAD(LEFT(N_FIRST_NAME_HEB,7), 7, " ") AS F_NAME_SHORT,
REPEAT(" ", 6) AS FILLER,
CASE
WHEN MP2OSEM = 1 or EXISTINGID = 1 THEN "2" 
ELSE " "
END AS UPDATETYPE,
REPEAT(" ", 10) AS FILLER1,
CONCAT(lpad(N_BIRTH_DATE_MON_YEAR, 2,"0"), LPAD(N_BIRTH_MONTH, 2, "0"), N_BIRTH_YEAR) AS BIRTH_DATE,
REPEAT(" ", 9) AS PERSONAL_NUM,
" " AS PASSPORT,
RPAD(LEFT(N_LAST_NAME_HEB,20), 20, " ") AS L_NAME_LONG,
RPAD(LEFT(N_FIRST_NAME_HEB,10), 10, " ") AS F_NAME_LONG,
REPEAT(" ", 10) AS FILLER3,
REPEAT(" ", 20) AS ENDSPACE
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW
WHERE EVENT200 = 1 OR MP2OSEM = 1
'''

## Event 201

In [0]:
#event 201
event201 = f'''
Select COMPANYNAME, "201", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE 
WHEN N_ACTION_TYPE = 'Z8' AND N_GROUP_ = 8 AND N_SUBGROUP IN ('I8', 'IC', 'IY', 'IA', 'IB')
OR N_ACTION_TYPE = 'Z8' AND N_GROUP_ = 9 AND N_SUBGROUP IN ('IX', 'IA') THEN "2"
WHEN N_ACTION_TYPE = 'Z3' OR EXISTINGID = 1 OR MP2OSEM = 1 THEN "2"
ELSE " "
END as UPDATE_TYPE, 
CASE 
WHEN STATUS = '00' OR substr(N_BEGDA_0000, 1, 2) <> "01" THEN SUBSTR(N_BEGDA_0000, 3, 6) 
ELSE SUBSTR(Prev_BEGDA_0000, 3,6)
END AS VALID_MONTH,
CASE 
WHEN MP2OSEM = 1 THEN '00' 
ELSE STATUS
END AS STATUS,
case
WHEN (STATUS = '00'  AND N_ACTION_TYPE = 'Z8')  OR MP2OSEM = 1 THEN CONCAT('18',SUBSTR(N_BEGDA_0000,3,6))
ELSE N_BEGDA_0000
END AS STATUS_DATE,
REPEAT(" ", 8) AS FILLER1,
"*" AS NEW,
REPEAT(" ", 63) AS SPACE_END
from {db_name}.STG_HILAN_ALL_CHANGES_VW
where (EVENT201=1 or MP2OSEM=1) and COMPANYNAME in ("OSEM", "PLSTNOSEM");
'''

## Event 205

In [0]:
# event 205
event205 = f'''
Select COMPANYNAME, "205", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE 
WHEN O_MARITAL_STATUS IS NULL AND CAST(O_MARITAL_STATUS_DATE AS INT) = 0 AND NVL(O_GENDER, "0") = '0' AND(N_MARITAL_STATUS IS NOT NULL OR CAST(N_MARITAL_STATUS_DATE AS INT) <> 0 OR NVL(N_GENDER, "0") <> '0') AND MP2OSEM =0 and EXISTINGID = 0 THEN ' '
ELSE '2'
END AS UPDATETYPE,
NVL(m.output, "0") as marital_status,
(nvl(DATE_FORMAT(TO_DATE(N_MARITAL_STATUS_DATE, 'yyyyMMdd'), 'ddMMyyyy'), "00000000")) AS MARITAL_STATUS_DATE,
REPEAT(" ", 40) AS FILLER1,
CASE 
WHEN O_PRNR IS NOT NULL AND MP2OSEM = 0 THEN substr(N_BEGDA_0002,3,6)
ELSE substr(N_BEGDA_0000,3,6)
END AS ENDVALIDDATE,
REPEAT(" ", 33) AS ENDSPACE
from {db_name}.STG_HILAN_ALL_CHANGES_VW s
left join {db_name}.dim_hilan_marital_status m on m.marital_status = s.n_marital_status and m.gender = s.n_gender
WHERE EVENT205=1 or MP2OSEM=1
'''

##Event 219

In [0]:
event219 = f'''
Select COMPANYNAME, "219", LPAD(PERSADMIN, 4, "0") as PERSADMIN, LPAD(EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE 
WHEN OLD_PRNR IS NOT NULL OR OLD_REF_MEMBER IS NOT NULL THEN '2'
ELSE " "
END AS UPDATETYPE,
LPAD(NVL(REF_PRNR_FAMILY_MEMBER, "0"), 9, "0") AS REF_PRNR_FAMILY_MEMBER,
NVL(DATE_FORMAT(TO_DATE(FAMILY_BIRTH_DATE, 'yyyyMMdd'), 'ddMMyyyy'), "00000000") AS BIRTH_DATE,
LEFT(RPAD(NVL(FAMILY_FIRST_NAME, " "), 10, " "), 10) AS PRIVATE_NAME,
REPEAT(" ", 16) AS FILLER1,
RELATION_CODE,
REPEAT(" ", 8) AS START_DATE,
REPEAT(" ", 36) AS ENDSPACE
from stg_hilan_multiple_changes_vw
WHERE EMP_ID IS NOT NULL
'''

## Event 221

In [0]:
event221 = f'''
WITH hilan_status_code as (Select COMPANYNAME, "221", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
" " as updatetype,
"0",
CASE 
WHEN N_ACTION_TYPE = "Z3" AND NOT (N_SUBGROUP IN ('I8', 'IC', 'IY', 'IA', 'IB') AND N_GROUP_ = '8' OR (N_GROUP_ = '9' AND N_SUBGROUP ='IA')) THEN "00"
WHEN N_ACTION_TYPE IN ("Z1" , "Z3") AND (N_SUBGROUP IN ('I8', 'IC', 'IY', 'IA', 'IB') AND N_GROUP_ = '8' OR (N_GROUP_ = '9' AND N_SUBGROUP ='IA')) THEN "89"
WHEN N_ACTION_TYPE='Z8' AND N_ACTION_REASON IN ('46', '05', '08', 'B4', '04', 'O4') AND NOT (N_SUBGROUP IN ('I8', 'IC', 'IY', 'IA', 'IB') AND N_GROUP_ = '8' OR (N_GROUP_ = '9' AND N_SUBGROUP ='IA')) THEN "43"
WHEN N_ACTION_TYPE='Z8' AND N_ACTION_REASON IN ('29','60','U3','44','25','18','19','20','56') AND NOT (N_SUBGROUP IN ('I8', 'IC', 'IY', 'IA', 'IB') AND N_GROUP_ = '8' OR (N_GROUP_ = '9' AND N_SUBGROUP ='IA')) THEN "44"
WHEN N_ACTION_TYPE='Z8' AND N_ACTION_REASON = '02' AND NOT (N_SUBGROUP IN ('I8', 'IC', 'IY', 'IA', 'IB') AND N_GROUP_ = '8' OR (N_GROUP_ = '9' AND N_SUBGROUP ='IA')) THEN "49"
WHEN N_ACTION_TYPE='Z8' AND (N_SUBGROUP IN ('I8', 'IC', 'IY', 'IA', 'IB') AND N_GROUP_ = '8' OR (N_GROUP_ = '9' AND N_SUBGROUP ='IA')) THEN "90"
END AS STATUS,
N_BEGDA_0000 as StartDate,
"99999999",
REPEAT(" ", 69) AS ENDSPACE
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW S 
WHERE EVENT221 = 1 AND COMPANYNAME = 'NESPRESSO' AND N_ACTION_TYPE IN ('Z8', 'Z1', 'Z3') ) 
select * from hilan_status_code 
where STATUS is not null;
'''

## Event 278

In [0]:
# event 278_OSEM
event278_osem = f'''
Select COMPANYNAME, 
"278", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE
WHEN O_FUNCTION_UNIT IS NULL AND O_POSITION IS NULL AND EXISTINGID = 0 AND (N_FUNCTION_UNIT IS NOT NULL OR N_POSITION IS NOT NULL AND N_EMP_ID IS NOT NULL) AND O_SUBAREA = '0' AND N_SUBAREA <> '0' AND UPDATETYPE = 'I' AND MP2OSEM = 0 THEN ' '
ELSE '2'
END AS UPDATETYPE,
REPEAT(" ", 3) AS FILLER1,
LPAD(RIGHT(N_SUBAREA, 3),3, "0") AS SUBAREA,
"000",
LPAD(NVL(N_FUNCTION_UNIT, " "), 9, " ") AS SUBBRANCH,
LPAD(N_PRNR, 9, " ") AS ORGANIZATIONUNIT,
REPEAT(" ", 10) AS FILLER2,
REPEAT(" ", 8) AS FILLER3,
CASE 
WHEN EVENT278_JUSTFUNCTIONUNIT=1 THEN CONCAT("01", DATE_FORMAT(TO_DATE(N_FUNC_UNIT_BEGDA, 'yyyyMMdd'), 'MMyyyy'))
WHEN MP2OSEM = 0 THEN CONCAT("01", substr(N_BEGDA_0001, 3,6))
ELSE CONCAT("01", substr(N_BEGDA_0000, 3,6))
END AS STARTDATE,
REPEAT(" ", 35) AS ENDSPACE
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW
WHERE (EVENT278_OSEM = 1 AND COMPANYNAME IN ('OSEM', 'PLSTNOSEM') AND N_ACTION_TYPE <> 'Z8') 
OR MP2OSEM = 1
'''


In [0]:
# event 278_NSP
event278_nsp = f'''
Select COMPANYNAME,"278", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE
WHEN EXISTINGID = 0 AND UPDATETYPE = 'I' AND MP2OSEM = 0 THEN ' '
ELSE '2'
END AS UPDATETYPE,
REPEAT(" ", 3) AS FILLER1,
repeat(" ", 3) AS SUBAREA,
LPAD(nvl(b.HILAN_BRANCH, "XXX"), 3,"0") as FIXED,
repeat(" ", 9) AS SUBBRANCH,
repeat(" ", 9) AS ORGANIZATIONUNIT,
REPEAT(" ", 10) AS FILLER2,
REPEAT(" ", 8) AS FILLER3,
CONCAT("01", substr(N_BEGDA_0001, 3,6)) as STARTDATE,
REPEAT(" ", 35) AS ENDSPACE
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW s
left join {db_name}.dim_hilan_branch b on b.SUB_AREA_CODE = s.n_subarea 
WHERE EVENT278_NSP = 1 AND COMPANYNAME = 'NESPRESSO' AND N_ACTION_TYPE <> 'Z8'
'''


## Event 218

In [0]:
# EVENT 218
event218 = f'''
Select COMPANYNAME, "218", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler,
CASE
WHEN O_ADDRESS_CITY_CODE IS NULL AND O_ADRESS_STREET_HOUSE = " "  AND  (N_ADDRESS_CITY_CODE IS NOT NULL OR N_ADRESS_STREET_HOUSE <> " ") and ExistingID = 0 and MP2OSEM = 0 THEN " "
ELSE "2"
END AS UPDATETYPE,
RPAD(NVL(RIGHT(N_ADDRESS_CITY_CODE,4), " "), 4, " ") AS CITYCODE,
REPEAT(" ", 13) AS FILLER1,
LEFT(LPAD(nvl(N_ADDRESS_POSTAL_CODE, " "), 7," "), 7) AS POSTALCODE,
LEFT(LPAD(NVL(CONCAT(N_ADRESS_STREET_HOUSE, " ", REVERSE(CAST(N_HOUSE_NUM AS VARCHAR(10)))), " "), 30, " "), 30) AS HOUSE_AND_STREET, 
RIGHT(LPAD(REGEXP_REPLACE(N_PRIVATE_PHONE, '[^a-zA-Z0-9]', ' '), 10, " "), 10) AS PHONENUMBER,
REPEAT(" ", 10) AS FILLER2,
CASE
WHEN MP2OSEM = 1 THEN CONCAT("18", substr(N_BEGDA_0000, 3,6))
WHEN MP2OSEM = 0 AND JUSTPHONE218 = 1 THEN  nvl(DATE_FORMAT(TO_DATE(N_PRIVATEPHONE_BEGDA_0105, 'yyyyMMdd'), 'ddMMyyyy'), "00000000")
ELSE N_BEGDA_0006
END AS STARTDATE,
REPEAT(" ", 6) AS ENDSPACE
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW
WHERE EVENT218=1 or MP2OSEM=1
'''

## Event 260

In [0]:
# EVENT 260
event260 = f'''
Select COMPANYNAME, "260", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
" " AS UPDATETYPE,
"01" AS EMPLOYMENT_PERIOD_TYPE,
CASE
WHEN MP2OSEM = 1 THEN CONCAT("18", substr(N_BEGDA_0000, 3,6))
ELSE N_BEGDA_0000
END AS EMPLOYMENT_PERIOD_DATE,
REPEAT(" ", 9) AS FILLER1,
CONCAT("01", substr(N_BEGDA_0000, 3,6)) AS VALIDATIONDATE, -- TO CHECK
REPEAT(" ", 61) AS ENDSPACE
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW 
WHERE EVENT260=1 or MP2OSEM=1
'''

## Event 274

In [0]:
event274 = f'''
Select COMPANYNAME, "274", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE
WHEN N_ACTION_TYPE = 'Z6' AND EXISTINGID = 0 THEN ' '
ELSE '2'
END AS UPDATETYPE, 
N_BEGDA_0000 as FROM_DATE,
"99999999" AS TILL_DATE,
REPEAT(" ", 6) AS FILLER1,
CASE
WHEN N_ACTION_TYPE = 'Z7' THEN '00'
WHEN COMPANYNAME <> 'NESPRESSO' AND N_ACTION_REASON = '01' THEN '85'
WHEN N_ACTION_REASON = '03' THEN '78'
WHEN COMPANYNAME <> 'NESPRESSO' AND N_ACTION_REASON = '04' THEN '86'
WHEN COMPANYNAME <> 'NESPRESSO' AND N_ACTION_REASON = '08' THEN '84'
WHEN COMPANYNAME <> 'NESPRESSO' AND N_ACTION_REASON = 'L1' THEN '81'
WHEN COMPANYNAME = 'NESPRESSO' AND N_ACTION_REASON = '01' THEN '74'
WHEN COMPANYNAME = 'NESPRESSO' AND N_ACTION_REASON = '04' THEN '75'
WHEN COMPANYNAME = 'NESPRESSO' AND N_ACTION_REASON = '08' THEN '89'
WHEN COMPANYNAME = 'NESPRESSO' AND N_ACTION_REASON = 'L1' THEN '79'
ELSE 'XX'
END AS ABSENCE_CODE,
REPEAT(" ", 64) AS ENDSPACE
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW 
WHERE EVENT274 = 1 
'''

## Event 279

In [0]:
event279=f'''
Select COMPANYNAME, "279", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE
WHEN MP2OSEM = 0 AND (O_GROUP_ IS NULL AND N_GROUP_ IS NOT NULL) AND (O_SUBGROUP IS NULL AND N_SUBGROUP IS NOT NULL) AND (O_POSITION IS NULL AND N_POSITION IS NOT NULL) AND EXISTINGID = 0 THEN ' ' 
ELSE '2'
END AS UPDATETYPE,
CASE 
WHEN MP2OSEM = 1 THEN '02' ---TO CHECK
WHEN N_GROUP_ = 9 AND N_SUBGROUP = 'IA' THEN '10'
WHEN N_GROUP_ = 9 AND N_SUBGROUP = 'ZY' THEN '04'
ELSE LPAD(C.status, 2, "0")
END AS STATUS,
CASE 
WHEN MP2OSEM = 0 THEN CONCAT("01", substr(N_BEGDA_0001, 3,6))
ELSE CONCAT("01", substr(N_BEGDA_0000, 3,6))
END AS STARTDATE, 
REPEAT(" ", 78) AS ENDSPACE
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW S
LEFT JOIN {db_name}.dim_hilan_contract_type_status C ON S.N_GROUP_ = C.GROUP_ AND S.N_SUBGROUP = C.SUBGROUP AND cast(s.Z1_z3_type as int) = cast(C.Z1_Z3 as int)
WHERE (EVENT279=1 or MP2OSEM=1) and COMPANYNAME IN ('OSEM', 'PLSTNOSEM')
'''

## Event 280

In [0]:
event280 =f'''
Select COMPANYNAME, "280", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE
WHEN MP2OSEM = 0 AND EXISTINGID = 0 AND O_EMP_PERCENTAGE IS NULL AND N_EMP_PERCENTAGE IS NOT NULL THEN " " ELSE "2" -- CHECK
END AS UPDATETYPE,
TO_CHAR(CAST(N_EMP_PERCENTAGE AS DOUBLE) * 100, "00000") AS EMP_PERC,
CASE
WHEN MP2OSEM =1 THEN CONCAT("18", substr(N_BEGDA_0000, 3,6)) 
ELSE N_BEGDA_0007
END AS STARTDATE,
"99999999" AS ENDDATE,
REPEAT(" ", 67) AS ENDSPACE 
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW
WHERE EVENT280=1 or MP2OSEM=1'''

## Event 298

In [0]:
event298 =f'''
Select COMPANYNAME, "298", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE
WHEN N_EMAIL IS NOT NULL AND O_EMAIL IS NULL AND EXISTINGID = 0 THEN ' ' 
ELSE '2'
END AS UPDATETYPE,
"1" AS ADDRESS_TYPE,
CASE 
WHEN COMPANYNAME = 'NESPRESSO' THEN RPAD(LEFT(LOWER(N_EMAIL),53),53, " ")
ELSE RPAD(LEFT(UPPER(N_EMAIL),53), 53, " ")
END AS EMAIL,
REPEAT(" ", 13) AS EMAIL2,
nvl(DATE_FORMAT(TO_DATE(N_EMAIL_BEGDA_0105, 'yyyyMMdd'), 'ddMMyyyy'), "00000000") AS START_DATE, 
"99999999" AS END_DATE
---REPEAT(" ", 10) AS ENDSPACE
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW
WHERE EVENT298=1 
'''

## Event 303

In [0]:
event303 = f'''
Select COMPANYNAME, "303", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE
WHEN MP2OSEM = 0 AND (O_BANK_KEY IS NULL AND N_BANK_KEY IS NOT NULL) AND (O_BANK_ACCOUNT_NUM IS NULL AND N_BANK_ACCOUNT_NUM IS NOT NULL) AND EXISTINGID = 0 THEN ' ' 
ELSE '2'
END AS UPDATETYPE,
"0",
NVL(SUBSTR(N_BANK_KEY, 1,2), "00") AS BANK_NUM,
NVL(RIGHT(N_BANK_KEY,3), "000") AS BANK_ACCOUNT,
LPAD(N_BANK_ACCOUNT_NUM, 9, "0") AS ACCOUNT_NUM,
REPEAT(" ", 73) AS ENDSPACE
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW S
WHERE n_bank_key is not null and
(EVENT303=1 or MP2OSEM=1)
'''

## Event 551

In [0]:
event551 = f'''
Select COMPANYNAME, "551", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE
WHEN  MP2OSEM = 0 AND EXISTINGID = 0 AND O_MIDDLE_NAME IS NULL AND O_LAST_NAME IS NULL AND O_FIRST_NAME IS NULL THEN ' ' 
ELSE '2'
END AS UPDATETYPE,
UPPER(RPAD(LEFT(N_FIRST_NAME,10), 10, " ")) AS F_NAME,
UPPER(RPAD(LEFT(N_MIDDLE_NAME,10), 10, " ")) AS M_NAME,
UPPER(RPAD(LEFT(N_LAST_NAME, 14), 14, " ")) AS L_NAME,
REPEAT(" ", 54) ENDSPACE
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW S
WHERE EVENT551=1 or MP2OSEM=1;'''

## Event 552

In [0]:
event552 = f'''
Select COMPANYNAME, "552", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
" " AS UPDATETYPE,
RPAD(LEFT(N_LAST_NAME_HEB,10), 10, " ") AS L_NAME_SHORT,
RPAD(LEFT(N_FIRST_NAME_HEB,7), 7, " ") AS F_NAME_SHORT,
RPAD(LEFT(N_LAST_NAME_HEB,20), 20, " ") AS L_NAME_LONG,
RPAD(LEFT(N_FIRST_NAME_HEB,10), 10, " ") AS F_NAME_LONG,
REPEAT(" ", 10) AS ADD_PR_NAME,
CASE
WHEN MP2OSEM =1 THEN CONCAT("18", substr(N_BEGDA_0000, 3,6)) 
ELSE N_BEGDA_0002
END AS ChangeDATE,
REPEAT(" ", 23) ENDSPACE
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW S
WHERE (EVENT552=1 or MP2OSEM=1) AND COMPANYNAME in ('OSEM', 'PLSTNOSEM');
'''

## Event 554

In [0]:
event554=f'''
Select COMPANYNAME, "554", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE
WHEN O_BIRTH_DATE_PASS IS NULL AND N_BIRTH_DATE_PASS IS NOT NULL and O_BIRTH_COUNTRY is null and N_BIRTH_COUNTRY IS NOT NULL AND MP2OSEM = 0 AND EXISTINGID = 0 THEN ' ' 
ELSE '2'
END AS UPDATETYPE,
CASE 
WHEN CAST(N_BIRTH_DATE_PASS AS INT) = 0 THEN "00000000"
ELSE DATE_FORMAT(TO_DATE(N_BIRTH_DATE_PASS, 'yyyyMMdd'), 'ddMMyyyy') 
END AS IMMIGRATION_DATE,
RPAD(nvl(C.HILAN_CODE, "0"), 3, " ") AS BIRTH_COUNTRY,
REPEAT(" ", 77) AS ENDSPACE
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW S
LEFT JOIN {db_name}.DIM_HILAN_COUNTRIES C ON S.N_BIRTH_COUNTRY = C.SAP_2_CODE
WHERE EVENT554=1 or MP2OSEM=1;'''

## Event 558

In [0]:
event558 =f'''
Select COMPANYNAME, "558", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE
WHEN O_JOB IS NULL AND N_JOB IS NOT NULL AND EXISTINGID = 0 IS NULL THEN " "
ELSE "2"
END AS UPDATETYPE,
LEFT(N_JOB, 6) AS JOB,
N_BEGDA_0001 AS FROM_DATE, 
REPEAT(" ", 74) AS ENDSPACE
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW S
WHERE EVENT558=1;'''

## Event 572 / 305

In [0]:
event572_305 = f'''
with event as (Select COMPANYNAME, "572", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
" " AS UPDATETYPE,
"305",
N_BEGDA_0000 AS FROM_DATE, 
REPEAT(" ", 8) AS FILLER1,
CASE
WHEN N_ACTION_TYPE = 'Z8' AND N_ACTION_REASON IN ('46', '05', '08', 'B4', '04', 'O4', 'J4', '10', '17', '44') THEN "000000021"
WHEN N_ACTION_TYPE = 'Z8' AND N_ACTION_REASON IN ('29','60', 'U3', '44', '25', '18', '19', '20', '56',  '30', '26') THEN "000000023"
WHEN N_ACTION_TYPE = 'Z8' AND N_ACTION_REASON = '41' OR N_ACTION_TYPE = 'ZA' AND N_ACTION_REASON = '04' THEN "000000025"
WHEN N_ACTION_TYPE = 'Z8' AND N_ACTION_REASON = '02' THEN "000000027"
WHEN N_ACTION_TYPE = 'Z8' AND N_ACTION_REASON IN ('45', 'Z9') THEN "000000026"
END AS END_OF_EMPLOYMENT,
REPEAT(" ", 60) AS ENDSPACE
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW S
WHERE EVENT572_305=1 and COMPANYNAME IN ('OSEM', 'PLSTNOSEM'))
select * from event where end_of_employment is not null;
'''


## Event 572 / 460

In [0]:
event572_460=f'''
Select COMPANYNAME, "572", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE
WHEN updatetype = "I" and EXISTINGID = 0 THEN " "
ELSE "2"
END AS UPDATE_TYPE,
"460" DATA_TYPE,
N_BEGDA_0001 AS STARTDATE,
"99999999" AS END_DATE,
LPAD(TRIM(N_prnr), 9, " ") as prnr,
REPEAT(" ",   60) AS ENDSPACE
from {db_name}.STG_HILAN_ALL_CHANGES_VW
where EVENT572_460=1 and companyname = "NESPRESSO";
'''

## Event 572 / 471

In [0]:
event572_471 =f'''
Select COMPANYNAME, "572", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE
WHEN O_JOB IS NULL AND N_JOB IS NOT NULL AND EXISTINGID = 0 AND MP2OSEM = 0 THEN " "
ELSE "2"
END AS UPDATE_TYPE,
"471" as DATA_TYPE,
CASE
WHEN MP2OSEM =1 THEN CONCAT("18", substr(N_BEGDA_0000, 3,6))
ELSE N_BEGDA_0001
END AS STARTDATE,
"99999999" AS END_DATE,
LPAD(N_JOB, 9, "0") as JOB,
REPEAT(" ",   60) AS ENDSPACE
from {db_name}.STG_HILAN_ALL_CHANGES_VW
where LPAD(N_JOB, 9, '0') <> "000000000"
and (EVENT572_471=1 or MP2OSEM=1)'''

## Event 572 / 541

In [0]:
event572_541 = f'''
Select COMPANYNAME, "572", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE
WHEN MP2OSEM = 0 AND O_EMP_ID IS NULL AND N_EMP_ID IS NOT NULL AND N_CP IS NOT NULL THEN " "
ELSE "2"
END AS UPDATE_TYPE,
"541",
CASE
WHEN MP2OSEM =1 THEN CONCAT("18", substr(N_BEGDA_0001, 3,6))
ELSE N_BEGDA_0001
END AS STARTDATE,
"99999999" AS END_DATE,
LPAD(N_CP, 9, "0"), 
REPEAT(" ",   60) AS END_SPACE
from {db_name}.STG_HILAN_ALL_CHANGES_VW
WHERE EVENT572_541=1 or MP2OSEM=1;
'''

## Event 572 / 560

In [0]:
#event 572/560
event572_560 = f'''
Select COMPANYNAME, "572", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE
WHEN  O_EMP_MANAGER IS NULL and N_EMP_MANAGER IS NOT NULL AND MP2OSEM = 0 AND EXISTINGID = 0 THEN " "
ELSE "2"
END AS UPDATE_TYPE,
CASE 
WHEN COMPANYNAME <> "NESPRESSO" THEN "560" 
ELSE "114"
END as DATA_TYPE,
CASE
WHEN MP2OSEM =1 THEN CONCAT("18", substr(N_BEGDA_0000,3,6))
WHEN COMPANYNAME in ('OSEM', 'PLSTNOSEM') AND N_ACTION_TYPE = 'Z5' AND to_date(N_BEGDA_0000, 'ddMMyyyy') > to_date(FILE_DATE_3_MONTHS, 'ddMMyyyy') THEN CONCAT("18", substr(PREV_BEGDA_0000,3,6))
WHEN COMPANYNAME in ('OSEM', 'PLSTNOSEM')  THEN CONCAT("18",PREV_MONTH)
ELSE CONCAT("01",CURR_MONTH)
END AS STARTDATE,
"99999999" AS END_DATE,
LPAD(nvl(f.emp_id,"0"), 9, "0") AS EMP_MANAGER,
REPEAT(" ", 9) AS FILLER1,
REPEAT(" ",   51) AS END_SPACE
from {db_name}.STG_HILAN_ALL_CHANGES_VW s
LEFT JOIN (select case when area <> 'ILN1' then 'OSEM' else 'NESPRESSO' end COMPANY, prnr, emp_id from {db_name}.stg_il_ee_sap_current) f on f.PRNR = s.n_emp_manager
WHERE (Event572_560=1 and N_ACTION_Type<>'Z8') or  MP2OSEM=1
'''

## Event 572 / 486

In [0]:
event572_486 =f'''
Select COMPANYNAME, "572", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, REPEAT(" ", 23) as Filler, 
CASE
WHEN N_PAY_TYPE is not null and O_PAY_TYPE IS NULL AND N_PAY_AREA is not null and O_PAY_AREA IS NULL AND N_PAY_GRADE is not null and O_PAY_GRADE IS NULL AND EXISTINGID = 0 AND MP2OSEM = 0 THEN " "
ELSE "2"
END AS UPDATE_TYPE,
"486",
N_BEGDA_0000 AS STARTDATE,
"99999999" AS END_DATE,
case
WHEN N_PAY_TYPE = 'MO' THEN '000000001'
WHEN N_PAY_TYPE = 'HO' THEN '000000002'
WHEN N_PAY_TYPE = 'HS' THEN '000000003'
WHEN N_PAY_TYPE = 'DR' THEN '000000004'
ELSE '000000000'
END AS PAY_TYPE,
case
WHEN N_PAY_AREA = 'NS' THEN '000000001'
WHEN N_PAY_AREA = 'SL' THEN '000000002'
ELSE '000000000'
END AS PAY_AREA,
LPAD(NVL(P.OUTPUT_VALUE, "0"), 9, "0") AS PAY_GRADE_GROUP,
REPEAT(" ",   42) AS ENDSPACE
from {db_name}.STG_HILAN_ALL_CHANGES_VW S
INNER JOIN {db_name}.dim_hilan_pay_types P ON P.INPUT_VALUE = S.N_PAY_GRADE
where n_pay_type in ('MO', 'HO', 'HS', 'DR') and n_pay_area in ('NS', 'SL')
and (EVENT572_486=1 AND N_ACTION_TYPE <> 'Z8' or MP2OSEM=1);
'''

## Event 800 / 150

In [0]:
event800_150 = f'''
Select COMPANYNAME, "800", LPAD(N_PERSADMIN, 4, "0") as N_PERSADMIN, LPAD(N_EMP_ID, 9, "0") AS ID_NUMBER, 
CASE
WHEN N_COST_CENTER is not null and O_COST_CENTER IS NULL AND EXISTINGID = 0 AND MP2OSEM = 0 THEN " "
ELSE "2"
END AS UPDATE_TYPE,
"150",
" " AS FILLER,
RPAD("1", 15, "0") AS KEY,
N_BEGDA_0001 AS STARTDATE,
REPEAT(" ", 8) AS END_DATE,
"1 79",
REPEAT(" ", 25) AS FILLER1,
LPAD(N_COST_CENTER, 10, "0") AS COST_CENTER,
REPEAT(" ", 37)
FROM {db_name}.STG_HILAN_ALL_CHANGES_VW
WHERE EVENT800_150=1 and COMPANYNAME<>"NESPRESSO";
'''

## Generate final file

In [0]:
glossary = {
  "Event200": event200,
  "Event201": event201,
  "Event205": event205,
  "Event218": event218,
  "Event219": event219,
  "Event221": event221,
  "Event260": event260,
  "Event274": event274,
  "Event278_OSEM": event278_osem,
  "Event278_NSP": event278_nsp,
  "Event279": event279,
  "Event280": event280,
  "Event298": event298,
  "Event303": event303,
  "Event551": event551,
  "Event552": event552,
  "Event554": event554,
  "Event558": event558,
  "Event572_560": event572_560,
  "Event572_305": event572_305,
  "Event572_460": event572_460,
  "Event572_471": event572_471,
  "Event572_486": event572_486,
  "Event572_541": event572_541,
  "Event572_560": event572_560,
  "Event800_150": event800_150
}

In [0]:
event_list = ['Event200', 'Event201', 'Event205', 'Event218', 'Event219', 'Event221', 'Event260', 'Event274', 'Event278_OSEM', 'Event278_NSP', 'Event279', 'Event280', 'Event298', 'Event303', 'Event551', 'Event552', 'Event554', 'Event558', 'Event572_305', 'Event572_471', 'Event572_486', 'Event572_460', 'Event572_541', 'Event572_560', 'Event800_150']

In [0]:
count_OSEM = 2          # according to HILAN requirements in documentation
count_PLSTNOSEM = 1
count_NESPRESSO = 1

col = StructType([StructField("merged", StringType(), True)])

main_OSEM = spark.createDataFrame([], col)
main_PLSTNOSEM = spark.createDataFrame([], col)
main_NESPRESSO = spark.createDataFrame([], col)

file_date = files[0][1].split("_")[4]+files[0][1].split("_")[5].split(".")[0]
print('Preparing files from ', file_date)

for event in event_list:
  print("--------------------------------------------------------------------")
  evdf = spark.sql(glossary[event])
  evdf = evdf.withColumn("merged", (concat_ws("", *evdf.columns[1:])))
  for company in ["OSEM", "NESPRESSO", "PLSTNOSEM"]:
    ev = evdf.filter(evdf["COMPANYNAME"]==company)
    if ev.count()>0:
      print(event, " - data found for", company)
      start_time = time.time()
      globals()[f"count_{company}"] = globals()[f"count_{company}"] +ev.count() 
      globals()[f"main_{company}"] = globals()[f"main_{company}"].union(ev.select('merged'))
      end_time = time.time()
      print(f"Execution time of {event} for {company}: {round((end_time - start_time), 2)} seconds")
    else:
      print("no data for", event, "for", company)

Preparing files from  202412312000
--------------------------------------------------------------------
Event200  - data found for OSEM
Execution time of Event200 for OSEM: 0.27 seconds
Event200  - data found for NESPRESSO
Execution time of Event200 for NESPRESSO: 0.23 seconds
no data for Event200 for PLSTNOSEM
--------------------------------------------------------------------
Event201  - data found for OSEM
Execution time of Event201 for OSEM: 0.24 seconds
no data for Event201 for NESPRESSO
no data for Event201 for PLSTNOSEM
--------------------------------------------------------------------
Event205  - data found for OSEM
Execution time of Event205 for OSEM: 0.42 seconds
Event205  - data found for NESPRESSO
Execution time of Event205 for NESPRESSO: 0.34 seconds
no data for Event205 for PLSTNOSEM
--------------------------------------------------------------------
Event218  - data found for OSEM
Execution time of Event218 for OSEM: 0.24 seconds
Event218  - data found for NESPRESSO


In [0]:
file_date = files[0][1].split("_")[4]+files[0][1].split("_")[5].split(".")[0]
for company in ["OSEM", "NESPRESSO", "PLSTNOSEM"]:
  if globals()[f"count_{company}"]>1:
    date = datetime.now().strftime("%Y%m%d%H%M%s")[:-8]
    count = str(globals()[f"count_{company}"]).rjust(14, " ")
    prefix = [(f'{date}****** HEAD OF FILE******',)]    # file header
    suffix = [(f'{count} ******* END OF FILE *****',)]    # file footer
    if company == "OSEM":
      final_file = spark.createDataFrame(prefix, col).union(globals()[f"main_{company}"]).union(spark.createDataFrame(suffix, col)) # dataframe with the final file
    else:
      final_file = (globals()[f"main_{company}"]).union(spark.createDataFrame(suffix, col)) # dataframe with the final file
    
    display(final_file)
  else:
    final_file = spark.createDataFrame([], col)
  if company == "NESPRESSO" :
    file_name = f"ILNSPTOHILAN.ILHRDATA{file_date}.txt"   #file name
  else:
    file_name = f"IL{company}TOHILAN.ILHRDATA{file_date}.dat"

  # saving the files in two steps in order to avoid UTF-8 encoding: 
  # step 1 :temporary location  
  dbutils.fs.rm(temp,  recurse=True) # cleaning the temp folder
  print("Saving the file to temporary location")
  final_file.coalesce(1).write.option("encoding", "windows-1255").option("ignoreLeadingWhiteSpace", "false").option("ignoreTrailingWhiteSpace", "false").option("quote", "\u0000").option("escape", "\u0000").mode("overwrite").csv(temp) # saving the file in Windows-1255 encoding
    
  # step 2: moving the file to permanent folder (output_location)
  temp_files = dbutils.fs.ls(temp)
  for each_file in temp_files:
    if each_file[1].startswith("part-"):
      print(f"Saving file {file_name} to {output_location} with number of lines:", (globals()[f"count_{company}"]-1))
      dbutils.fs.cp(each_file[0], (output_location+file_name))
      print("------------------------------------------------------")
  

merged
20250102130817****** HEAD OF FILE******
2000240022588313יצחק רינה עד 05051967 יצחק רינה עדיה
2000210011963097עבד אללה סחר 06071964 עבד אללה סחר
2000244322827916סנדקלי עודאי 03072001 סנדקלי עודאי
2000211033323833ועקנין אסף 29091976 ועקנין אסף
2000244318980638עווד עדנאן 08011998 עווד עדנאן
2000202346349137מנגסטו בקלה 17051964 מנגסטו בקלה
2000203311478457אמארה מהא 06121993 אמארה מהא
2000210022797930גובני מרגלית 2 11111966 גובני מרגלית
2000210032336737ויזמן רחמים 30031975 ויזמן רחמים


Saving the file to temporary location
Saving file ILOSEMTOHILAN.ILHRDATA202412312000.dat to dbfs:/mnt/prod/inbound/datastage/hr/output/ with number of lines: 819
------------------------------------------------------


merged
2002282304947047כהן חן 10051990 כהן חן
2002282209227586בן שושן דניאל 06111999 בן שושן דניאל
2002282039664776וולנובסקי איל 03091984 וולנובסקי איל
2002282066351727ורולקר שלמה אל 06111983 ורולקר שלמה אליהו
2002282043249325עמר גל-לירו 16091981 עמר גל-לירון
2002282305788457כבהא זייד 13011993 כבהא זייד
2052282209227586 100000000 102024
2052282066351727 100000000 012025
2052282043249325 500000000 012025
2052282305788457 100000000 012025


Saving the file to temporary location
Saving file ILNSPTOHILAN.ILHRDATA202412312000.txt to dbfs:/mnt/prod/inbound/datastage/hr/output/ with number of lines: 70
------------------------------------------------------
Saving the file to temporary location
Saving file ILPLSTNOSEMTOHILAN.ILHRDATA202412312000.dat to dbfs:/mnt/prod/inbound/datastage/hr/output/ with number of lines: 0
------------------------------------------------------


## Update of tables

In [0]:
# update of tables with yesterday data
stmt = f'''
CREATE OR REPLACE TABLE {db_name}.STG_HILAN_FUT_CURR_YESTERDAY as
select * from (
select 'FUTURE' DATATYPE, N_PRNR
from STG_HILAN_FUTURE_CURR_VW
union all
select 'CURR' DATATYPE, PRNR
from FutureNew
)a
where not exists ( select 1 
from STG_HILAN_CURR_NEW_OLD_VW b 
where b.N_PRNR=a.N_PRNR)
'''
spark.sql(stmt)

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]

In [0]:
# update of tables with new data
for file_type in ["CURRENT", "FUTURE", "MULTIPLE"]:
  spark.sql(f"DROP TABLE IF EXISTS {db_name}.stg_il_ee_sap_{file_type};")
  globals()[f"df_{file_type}"].write.format("delta").option("mergeSchema", "true").saveAsTable(db_name+'.stg_il_ee_sap_'+file_type)
  stmt = f"create table if not exists {db_name}.stg_il_ee_sap_{file_type} using delta location 'dbfs:/mnt/{container}/solutions/HR/STG_IL_EE_SAP_{file_type}'"

In [0]:
# moving to archive
for file in files:
  dbutils.fs.mv(file[0], file[0].replace('inbound', 'archive'))